In [1]:
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 경로 참 더럽네
from CRNNKerasmaster.parameter import *
from CRNNKerasmaster.Model import get_Model
from CRNNKerasmaster.Image_Generator import TextImageGenerator

import cv2
import itertools, os, time
import numpy as np
import argparse
import easydict
from datetime import datetime

K.set_learning_phase(0)

2022-12-19 15:54:03.358446: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 15:54:03.539535: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 15:54:03.539584: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-19 15:54:04.300969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
model = get_Model(training=True)

2022-12-19 15:54:05.438687: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ghwns82/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-19 15:54:05.438775: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-19 15:54:05.438848: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-HN4KDSG): /proc/driver/nvidia/version does not exist
2022-12-19 15:54:05.439107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate 

In [4]:
# try:
#     model.load_weights('LSTM+BN4--26--0.011.hdf5')
#     print("...Previous weight data...")
# except:
#     print("...New weight data...")
#     pass

In [5]:
train_file_path = './CRNNKerasmaster/DB/train_9/'
tiger_train = TextImageGenerator(train_file_path, img_w, img_h, batch_size, downsample_factor)
tiger_train.build_data()

39  Image Loading start...
True
39  Image Loading finish...


In [6]:
valid_file_path = './CRNNKerasmaster/DB/test_9/'
tiger_val = TextImageGenerator(valid_file_path, img_w, img_h, val_batch_size, downsample_factor)
tiger_val.build_data()

3  Image Loading start...
True
3  Image Loading finish...


In [7]:
# tf.config.run_functions_eagerly(True)

In [8]:
time_str='%Y_%m_%d_%H_%M_%S'

ada = Adadelta()

early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=4, mode='min', verbose=1)
checkpoint = ModelCheckpoint(filepath='LSTM+BN5--text9--{epoch:02d}--{val_loss:.3f}.hdf5', monitor='loss', verbose=10, mode='min', period=1000)
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

In [9]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1/1 [==============================] - 7s 7s/step - loss: 193.8440 - val_loss: 193.9976
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 193.0020 - val_loss: 193.9285
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 194.1773 - val_loss: 192.6805
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 189.5901 - val_loss: 192.7170
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 192.9405 - val_loss: 193.1183
Epoch 6/1000
1/1 [==============================] - 1s 1s/step - loss: 189.2179 - val_loss: 193.5739
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 191.0151 - val_loss: 193.4565
Epoch 8/1000
1/1 [==============================] - 1s 1s/step - loss: 187.8540 - val_loss: 

1/1 [==============================] - 1s 1s/step - loss: 136.3930 - val_loss: 176.7508
Epoch 77/1000
1/1 [==============================] - 1s 1s/step - loss: 129.2265 - val_loss: 172.3572
Epoch 78/1000
1/1 [==============================] - 1s 1s/step - loss: 130.2044 - val_loss: 172.0839
Epoch 79/1000
1/1 [==============================] - 1s 1s/step - loss: 128.7742 - val_loss: 171.8174
Epoch 80/1000
1/1 [==============================] - 1s 1s/step - loss: 129.3129 - val_loss: 171.5107
Epoch 81/1000
1/1 [==============================] - 1s 1s/step - loss: 144.1728 - val_loss: 171.2917
Epoch 82/1000
1/1 [==============================] - 1s 1s/step - loss: 127.2165 - val_loss: 174.5994
Epoch 83/1000
1/1 [==============================] - 2s 2s/step - loss: 131.9917 - val_loss: 180.3931
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 126.5498 - val_loss: 170.4901
Epoch 85/1000
1/1 [==============================] - 1s 1s/step - loss: 128.8635 - val_loss: 173

1/1 [==============================] - 1s 1s/step - loss: 90.2425 - val_loss: 151.9577
Epoch 157/1000
1/1 [==============================] - 2s 2s/step - loss: 97.6364 - val_loss: 151.8392
Epoch 158/1000
1/1 [==============================] - 1s 1s/step - loss: 107.3871 - val_loss: 155.2274
Epoch 159/1000
1/1 [==============================] - 2s 2s/step - loss: 99.1385 - val_loss: 151.5336
Epoch 160/1000
1/1 [==============================] - 1s 1s/step - loss: 110.2967 - val_loss: 151.3866
Epoch 161/1000
1/1 [==============================] - 2s 2s/step - loss: 99.1656 - val_loss: 151.2344
Epoch 162/1000
1/1 [==============================] - 1s 1s/step - loss: 92.0742 - val_loss: 167.3255
Epoch 163/1000
1/1 [==============================] - 1s 1s/step - loss: 89.2589 - val_loss: 154.6417
Epoch 164/1000
1/1 [==============================] - 1s 1s/step - loss: 86.6903 - val_loss: 167.0821
Epoch 165/1000
1/1 [==============================] - 1s 1s/step - loss: 92.1506 - val_loss: 15

1/1 [==============================] - 1s 1s/step - loss: 79.4864 - val_loss: 158.7081
Epoch 237/1000
1/1 [==============================] - 1s 1s/step - loss: 83.5585 - val_loss: 158.4579
Epoch 238/1000
1/1 [==============================] - 1s 1s/step - loss: 84.2475 - val_loss: 158.1807
Epoch 239/1000
1/1 [==============================] - 1s 1s/step - loss: 78.1903 - val_loss: 145.5496
Epoch 240/1000
1/1 [==============================] - 1s 1s/step - loss: 70.2748 - val_loss: 148.4123
Epoch 241/1000
1/1 [==============================] - 1s 1s/step - loss: 69.5849 - val_loss: 148.3841
Epoch 242/1000
1/1 [==============================] - 1s 1s/step - loss: 70.5866 - val_loss: 145.4536
Epoch 243/1000
1/1 [==============================] - 2s 2s/step - loss: 79.4444 - val_loss: 157.4466
Epoch 244/1000
1/1 [==============================] - 1s 1s/step - loss: 84.1056 - val_loss: 157.2799
Epoch 245/1000
1/1 [==============================] - 1s 1s/step - loss: 71.9513 - val_loss: 157.

1/1 [==============================] - 2s 2s/step - loss: 63.7442 - val_loss: 147.6799
Epoch 317/1000
1/1 [==============================] - 1s 1s/step - loss: 60.9564 - val_loss: 144.4778
Epoch 318/1000
1/1 [==============================] - 2s 2s/step - loss: 62.7973 - val_loss: 144.3272
Epoch 319/1000
1/1 [==============================] - 2s 2s/step - loss: 60.1965 - val_loss: 144.2548
Epoch 320/1000
1/1 [==============================] - 2s 2s/step - loss: 67.6562 - val_loss: 152.4328
Epoch 321/1000
1/1 [==============================] - 2s 2s/step - loss: 68.3200 - val_loss: 147.4211
Epoch 322/1000
1/1 [==============================] - 2s 2s/step - loss: 59.9589 - val_loss: 147.1126
Epoch 323/1000
1/1 [==============================] - 2s 2s/step - loss: 61.9666 - val_loss: 144.0772
Epoch 324/1000
1/1 [==============================] - 2s 2s/step - loss: 59.9156 - val_loss: 147.0457
Epoch 325/1000
1/1 [==============================] - 2s 2s/step - loss: 67.4261 - val_loss: 143.

...

1/1 [==============================] - 2s 2s/step - loss: 37.0171 - val_loss: 139.0868
Epoch 637/1000
1/1 [==============================] - 2s 2s/step - loss: 37.6281 - val_loss: 160.8531
Epoch 638/1000
1/1 [==============================] - 2s 2s/step - loss: 39.7660 - val_loss: 138.8559
Epoch 639/1000
1/1 [==============================] - 1s 1s/step - loss: 41.0244 - val_loss: 160.7380
Epoch 640/1000
1/1 [==============================] - 1s 1s/step - loss: 39.7894 - val_loss: 148.0643
Epoch 641/1000
1/1 [==============================] - 1s 1s/step - loss: 35.5731 - val_loss: 160.5653
Epoch 642/1000
1/1 [==============================] - 1s 1s/step - loss: 36.6890 - val_loss: 138.8625
Epoch 643/1000
1/1 [==============================] - 2s 2s/step - loss: 37.9008 - val_loss: 160.8339
Epoch 644/1000
1/1 [==============================] - 1s 1s/step - loss: 39.6583 - val_loss: 150.9615
Epoch 645/1000
1/1 [==============================] - 2s 2s/step - loss: 37.2247 - val_loss: 160.

1/1 [==============================] - 1s 1s/step - loss: 35.4338 - val_loss: 148.0409
Epoch 717/1000
1/1 [==============================] - 1s 1s/step - loss: 34.1999 - val_loss: 147.8752
Epoch 718/1000
1/1 [==============================] - 2s 2s/step - loss: 36.2987 - val_loss: 137.7443
Epoch 719/1000
1/1 [==============================] - 1s 1s/step - loss: 36.8304 - val_loss: 147.8695
Epoch 720/1000
1/1 [==============================] - 1s 1s/step - loss: 42.3099 - val_loss: 137.8115
Epoch 721/1000
1/1 [==============================] - 2s 2s/step - loss: 48.0301 - val_loss: 137.5725
Epoch 722/1000
1/1 [==============================] - 1s 1s/step - loss: 34.5690 - val_loss: 147.5705
Epoch 723/1000
1/1 [==============================] - 1s 1s/step - loss: 34.3297 - val_loss: 147.5608
Epoch 724/1000
1/1 [==============================] - 2s 2s/step - loss: 33.0155 - val_loss: 137.7663
Epoch 725/1000
1/1 [==============================] - 2s 2s/step - loss: 31.3378 - val_loss: 160.

1/1 [==============================] - 2s 2s/step - loss: 43.5620 - val_loss: 159.6488
Epoch 797/1000
1/1 [==============================] - 2s 2s/step - loss: 33.2228 - val_loss: 159.8420
Epoch 798/1000
1/1 [==============================] - 2s 2s/step - loss: 31.0504 - val_loss: 135.4267
Epoch 799/1000
1/1 [==============================] - 2s 2s/step - loss: 36.6476 - val_loss: 135.2840
Epoch 800/1000
1/1 [==============================] - 2s 2s/step - loss: 32.5147 - val_loss: 135.3367
Epoch 801/1000
1/1 [==============================] - 2s 2s/step - loss: 31.5578 - val_loss: 135.2169
Epoch 802/1000
1/1 [==============================] - 2s 2s/step - loss: 28.3990 - val_loss: 159.6676
Epoch 803/1000
1/1 [==============================] - 2s 2s/step - loss: 29.3922 - val_loss: 135.3300
Epoch 804/1000
1/1 [==============================] - 2s 2s/step - loss: 40.2950 - val_loss: 135.3103
Epoch 805/1000
1/1 [==============================] - 2s 2s/step - loss: 45.1302 - val_loss: 145.

1/1 [==============================] - 1s 1s/step - loss: 24.7875 - val_loss: 134.1430
Epoch 877/1000
1/1 [==============================] - 2s 2s/step - loss: 25.9941 - val_loss: 159.1059
Epoch 878/1000
1/1 [==============================] - 2s 2s/step - loss: 30.3481 - val_loss: 145.7510
Epoch 879/1000
1/1 [==============================] - 2s 2s/step - loss: 29.1131 - val_loss: 159.3847
Epoch 880/1000
1/1 [==============================] - 2s 2s/step - loss: 25.8780 - val_loss: 134.2359
Epoch 881/1000
1/1 [==============================] - 1s 1s/step - loss: 28.8844 - val_loss: 134.3120
Epoch 882/1000
1/1 [==============================] - 1s 1s/step - loss: 25.7604 - val_loss: 145.9754
Epoch 883/1000
1/1 [==============================] - 2s 2s/step - loss: 31.6108 - val_loss: 159.4662
Epoch 884/1000
1/1 [==============================] - 2s 2s/step - loss: 28.5683 - val_loss: 159.1870
Epoch 885/1000
1/1 [==============================] - 2s 2s/step - loss: 27.7107 - val_loss: 134.

1/1 [==============================] - 2s 2s/step - loss: 26.9497 - val_loss: 170.7003
Epoch 957/1000
1/1 [==============================] - 2s 2s/step - loss: 31.7308 - val_loss: 131.5215
Epoch 958/1000
1/1 [==============================] - 1s 1s/step - loss: 22.4257 - val_loss: 144.0015
Epoch 959/1000
1/1 [==============================] - 1s 1s/step - loss: 22.9333 - val_loss: 157.9091
Epoch 960/1000
1/1 [==============================] - 2s 2s/step - loss: 24.4492 - val_loss: 131.6794
Epoch 961/1000
1/1 [==============================] - 1s 1s/step - loss: 28.7875 - val_loss: 157.8803
Epoch 962/1000
1/1 [==============================] - 2s 2s/step - loss: 34.7891 - val_loss: 131.4496
Epoch 963/1000
1/1 [==============================] - 1s 1s/step - loss: 38.7493 - val_loss: 158.3585
Epoch 964/1000
1/1 [==============================] - 2s 2s/step - loss: 24.7296 - val_loss: 158.1820
Epoch 965/1000
1/1 [==============================] - 1s 1s/step - loss: 21.4528 - val_loss: 131.

In [10]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 26.5333 - val_loss: 117.3022
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 21.6446 - val_loss: 143.5931
Epoch 3/1000
1/1 [==============================] - 1s 1s/step - loss: 20.3048 - val_loss: 157.6858
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 33.6568 - val_loss: 131.4152
Epoch 5/1000
1/1 [==============================] - 1s 1s/step - loss: 28.1194 - val_loss: 143.2624
Epoch 6/1000
1/1 [==============================] - 1s 1s/step - loss: 22.5763 - val_loss: 143.1457
Epoch 7/1000
1/1 [==============================] - 1s 1s/step - loss: 23.1596 - val_loss: 169.1472
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 20.8244 - val_loss: 157.2800
Epoch 9/1000
1/1 [==============================] - 1s 1s/step - loss: 23.6166 - val_loss: 131.0950
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 21.1123 - val_loss: 157.3061

1/1 [==============================] - 1s 1s/step - loss: 28.9550 - val_loss: 129.1505
Epoch 83/1000
1/1 [==============================] - 1s 1s/step - loss: 50.3574 - val_loss: 157.5990
Epoch 84/1000
1/1 [==============================] - 1s 1s/step - loss: 26.2101 - val_loss: 157.3868
Epoch 85/1000
1/1 [==============================] - 1s 1s/step - loss: 20.4648 - val_loss: 113.7965
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 19.1560 - val_loss: 157.3471
Epoch 87/1000
1/1 [==============================] - 1s 1s/step - loss: 19.7166 - val_loss: 129.0543
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 18.9296 - val_loss: 128.7682
Epoch 89/1000
1/1 [==============================] - 1s 1s/step - loss: 21.3315 - val_loss: 141.8024
Epoch 90/1000
1/1 [==============================] - 1s 1s/step - loss: 20.1719 - val_loss: 141.6263
Epoch 91/1000
1/1 [==============================] - 1s 1s/step - loss: 17.3448 - val_loss: 169.7293
Epoc

1/1 [==============================] - 1s 1s/step - loss: 26.5948 - val_loss: 127.8450
Epoch 163/1000
1/1 [==============================] - 1s 1s/step - loss: 16.2974 - val_loss: 156.0221
Epoch 164/1000
1/1 [==============================] - 2s 2s/step - loss: 17.9126 - val_loss: 127.8135
Epoch 165/1000
1/1 [==============================] - 1s 1s/step - loss: 17.5374 - val_loss: 127.8376
Epoch 166/1000
1/1 [==============================] - 1s 1s/step - loss: 22.6544 - val_loss: 112.6522
Epoch 167/1000
1/1 [==============================] - 1s 1s/step - loss: 19.8272 - val_loss: 127.9258
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 19.1053 - val_loss: 141.0593
Epoch 169/1000
1/1 [==============================] - 1s 1s/step - loss: 14.9635 - val_loss: 140.9112
Epoch 170/1000
1/1 [==============================] - 1s 1s/step - loss: 16.3842 - val_loss: 156.0078
Epoch 171/1000
1/1 [==============================] - 1s 1s/step - loss: 26.4844 - val_loss: 140.

1/1 [==============================] - 1s 1s/step - loss: 13.9244 - val_loss: 126.8386
Epoch 243/1000
1/1 [==============================] - 1s 1s/step - loss: 14.5604 - val_loss: 126.8607
Epoch 244/1000
1/1 [==============================] - 2s 2s/step - loss: 16.9536 - val_loss: 142.5835
Epoch 245/1000
1/1 [==============================] - 1s 1s/step - loss: 15.3705 - val_loss: 155.2622
Epoch 246/1000
1/1 [==============================] - 2s 2s/step - loss: 17.3301 - val_loss: 155.1158
Epoch 247/1000
1/1 [==============================] - 1s 1s/step - loss: 18.5135 - val_loss: 155.3406
Epoch 248/1000
1/1 [==============================] - 1s 1s/step - loss: 14.8259 - val_loss: 126.9479
Epoch 249/1000
1/1 [==============================] - 1s 1s/step - loss: 20.1790 - val_loss: 139.8356
Epoch 250/1000
1/1 [==============================] - 1s 1s/step - loss: 20.8868 - val_loss: 155.3669
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 14.5554 - val_loss: 155.

1/1 [==============================] - 1s 1s/step - loss: 27.1873 - val_loss: 168.4706
Epoch 323/1000
1/1 [==============================] - 1s 1s/step - loss: 14.1473 - val_loss: 138.5459
Epoch 324/1000
1/1 [==============================] - 2s 2s/step - loss: 14.7493 - val_loss: 155.0522
Epoch 325/1000
1/1 [==============================] - 1s 1s/step - loss: 13.8656 - val_loss: 138.2798
Epoch 326/1000
1/1 [==============================] - 1s 1s/step - loss: 12.5526 - val_loss: 154.9488
Epoch 327/1000
1/1 [==============================] - 2s 2s/step - loss: 12.8804 - val_loss: 138.2532
Epoch 328/1000
1/1 [==============================] - 1s 1s/step - loss: 22.7468 - val_loss: 155.0222
Epoch 329/1000
1/1 [==============================] - 1s 1s/step - loss: 27.2989 - val_loss: 154.2440
Epoch 330/1000
1/1 [==============================] - 1s 1s/step - loss: 19.4133 - val_loss: 124.8472
Epoch 331/1000
1/1 [==============================] - 1s 1s/step - loss: 12.6099 - val_loss: 141.

...

Epoch 643/1000
1/1 [==============================] - 1s 1s/step - loss: 7.8419 - val_loss: 167.9012
Epoch 644/1000
1/1 [==============================] - 1s 1s/step - loss: 11.2608 - val_loss: 121.0235
Epoch 645/1000
1/1 [==============================] - 1s 1s/step - loss: 10.3027 - val_loss: 136.4066
Epoch 646/1000
1/1 [==============================] - 2s 2s/step - loss: 9.3218 - val_loss: 120.7928
Epoch 647/1000
1/1 [==============================] - 1s 1s/step - loss: 9.8432 - val_loss: 152.4230
Epoch 648/1000
1/1 [==============================] - 1s 1s/step - loss: 10.7245 - val_loss: 152.3294
Epoch 649/1000
1/1 [==============================] - 2s 2s/step - loss: 7.1540 - val_loss: 136.5975
Epoch 650/1000
1/1 [==============================] - 1s 1s/step - loss: 7.8520 - val_loss: 152.2977
Epoch 651/1000
1/1 [==============================] - 1s 1s/step - loss: 8.4786 - val_loss: 152.3279
Epoch 652/1000
1/1 [==============================] - 1s 1s/step - loss: 9.8689 - val_lo

Epoch 724/1000
1/1 [==============================] - 2s 2s/step - loss: 28.3466 - val_loss: 135.6347
Epoch 725/1000
1/1 [==============================] - 2s 2s/step - loss: 10.8402 - val_loss: 119.8625
Epoch 726/1000
1/1 [==============================] - 1s 1s/step - loss: 8.4054 - val_loss: 135.4640
Epoch 727/1000
1/1 [==============================] - 1s 1s/step - loss: 11.3145 - val_loss: 151.6851
Epoch 728/1000
1/1 [==============================] - 1s 1s/step - loss: 8.2690 - val_loss: 135.3980
Epoch 729/1000
1/1 [==============================] - 1s 1s/step - loss: 11.6592 - val_loss: 119.9773
Epoch 730/1000
1/1 [==============================] - 1s 1s/step - loss: 9.0679 - val_loss: 103.4346
Epoch 731/1000
1/1 [==============================] - 1s 1s/step - loss: 8.6240 - val_loss: 151.5598
Epoch 732/1000
1/1 [==============================] - 1s 1s/step - loss: 6.2823 - val_loss: 119.7457
Epoch 733/1000
1/1 [==============================] - 1s 1s/step - loss: 7.1269 - val_l

Epoch 805/1000
1/1 [==============================] - 1s 1s/step - loss: 6.1298 - val_loss: 150.3237
Epoch 806/1000
1/1 [==============================] - 2s 2s/step - loss: 6.8381 - val_loss: 134.7007
Epoch 807/1000
1/1 [==============================] - 1s 1s/step - loss: 7.0687 - val_loss: 150.3354
Epoch 808/1000
1/1 [==============================] - 1s 1s/step - loss: 6.7320 - val_loss: 118.9728
Epoch 809/1000
1/1 [==============================] - 1s 1s/step - loss: 6.6617 - val_loss: 118.8141
Epoch 810/1000
1/1 [==============================] - 1s 1s/step - loss: 6.5867 - val_loss: 150.4173
Epoch 811/1000
1/1 [==============================] - 1s 1s/step - loss: 8.9956 - val_loss: 150.6354
Epoch 812/1000
1/1 [==============================] - 1s 1s/step - loss: 10.0098 - val_loss: 134.8167
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 7.0030 - val_loss: 118.7299
Epoch 814/1000
1/1 [==============================] - 1s 1s/step - loss: 7.5241 - val_loss

1/1 [==============================] - 2s 2s/step - loss: 6.6985 - val_loss: 133.3383
Epoch 887/1000
1/1 [==============================] - 2s 2s/step - loss: 5.8770 - val_loss: 133.8416
Epoch 888/1000
1/1 [==============================] - 1s 1s/step - loss: 5.2469 - val_loss: 117.8652
Epoch 889/1000
1/1 [==============================] - 1s 1s/step - loss: 6.6260 - val_loss: 148.7747
Epoch 890/1000
1/1 [==============================] - 1s 1s/step - loss: 39.9819 - val_loss: 117.0622
Epoch 891/1000
1/1 [==============================] - 2s 2s/step - loss: 18.7366 - val_loss: 148.1660
Epoch 892/1000
1/1 [==============================] - 2s 2s/step - loss: 6.8706 - val_loss: 164.7395
Epoch 893/1000
1/1 [==============================] - 2s 2s/step - loss: 8.8729 - val_loss: 148.1682
Epoch 894/1000
1/1 [==============================] - 1s 1s/step - loss: 5.6933 - val_loss: 116.8566
Epoch 895/1000
1/1 [==============================] - 1s 1s/step - loss: 5.1422 - val_loss: 132.0039
Epo

1/1 [==============================] - 1s 1s/step - loss: 10.3771 - val_loss: 132.6384
Epoch 968/1000
1/1 [==============================] - 2s 2s/step - loss: 29.3838 - val_loss: 148.1337
Epoch 969/1000
1/1 [==============================] - 1s 1s/step - loss: 6.0831 - val_loss: 148.1079
Epoch 970/1000
1/1 [==============================] - 2s 2s/step - loss: 6.1890 - val_loss: 116.2227
Epoch 971/1000
1/1 [==============================] - 1s 1s/step - loss: 5.4335 - val_loss: 148.0455
Epoch 972/1000
1/1 [==============================] - 2s 2s/step - loss: 4.9614 - val_loss: 148.0051
Epoch 973/1000
1/1 [==============================] - 1s 1s/step - loss: 6.1992 - val_loss: 116.3291
Epoch 974/1000
1/1 [==============================] - 1s 1s/step - loss: 6.5400 - val_loss: 147.6502
Epoch 975/1000
1/1 [==============================] - 1s 1s/step - loss: 7.4206 - val_loss: 116.1022
Epoch 976/1000
1/1 [==============================] - 1s 1s/step - loss: 5.8063 - val_loss: 132.9534
Epo

In [11]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 7.0471 - val_loss: 147.4105
Epoch 2/1000
1/1 [==============================] - 1s 1s/step - loss: 39.7474 - val_loss: 116.6267
Epoch 3/1000
1/1 [==============================] - 1s 1s/step - loss: 23.7294 - val_loss: 116.4668
Epoch 4/1000
1/1 [==============================] - 1s 1s/step - loss: 5.1422 - val_loss: 116.3945
Epoch 5/1000
1/1 [==============================] - 1s 1s/step - loss: 5.4986 - val_loss: 132.4150
Epoch 6/1000
1/1 [==============================] - 1s 1s/step - loss: 8.2207 - val_loss: 100.9808
Epoch 7/1000
1/1 [==============================] - 1s 1s/step - loss: 6.5886 - val_loss: 132.4638
Epoch 8/1000
1/1 [==============================] - 1s 1s/step - loss: 5.7516 - val_loss: 115.9188
Epoch 9/1000
1/1 [==============================] - 1s 1s/step - loss: 5.2913 - val_loss: 115.9741
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 4.4426 - val_loss: 115.9320
Epoch 1

Epoch 83/1000
1/1 [==============================] - 1s 1s/step - loss: 4.4526 - val_loss: 114.9157
Epoch 84/1000
1/1 [==============================] - 1s 1s/step - loss: 4.5437 - val_loss: 99.9000
Epoch 85/1000
1/1 [==============================] - 1s 1s/step - loss: 4.6769 - val_loss: 146.7228
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 9.3650 - val_loss: 146.6800
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 4.8064 - val_loss: 163.4380
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 5.3541 - val_loss: 114.7428
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 4.8749 - val_loss: 146.7374
Epoch 90/1000
1/1 [==============================] - 1s 1s/step - loss: 7.6574 - val_loss: 99.5838
Epoch 91/1000
1/1 [==============================] - 1s 1s/step - loss: 5.5989 - val_loss: 131.5651
Epoch 92/1000
1/1 [==============================] - 1s 1s/step - loss: 7.7206 - val_loss: 146.5457
Ep

1/1 [==============================] - 1s 1s/step - loss: 4.6509 - val_loss: 145.6152
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 4.2329 - val_loss: 131.0862
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 4.1576 - val_loss: 145.6652
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 4.2105 - val_loss: 114.4161
Epoch 168/1000
1/1 [==============================] - 1s 1s/step - loss: 4.9482 - val_loss: 128.6663
Epoch 169/1000
1/1 [==============================] - 1s 1s/step - loss: 4.7280 - val_loss: 145.4502
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 6.8627 - val_loss: 114.2630
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 3.8144 - val_loss: 99.7036
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 3.6254 - val_loss: 145.6158
Epoch 173/1000
1/1 [==============================] - 1s 1s/step - loss: 4.8426 - val_loss: 131.0550
Epoch 

1/1 [==============================] - 2s 2s/step - loss: 3.9638 - val_loss: 113.4421
Epoch 246/1000
1/1 [==============================] - 1s 1s/step - loss: 4.2596 - val_loss: 145.0695
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 5.8174 - val_loss: 112.8304
Epoch 248/1000
1/1 [==============================] - 1s 1s/step - loss: 4.3787 - val_loss: 144.7527
Epoch 249/1000
1/1 [==============================] - 1s 1s/step - loss: 5.0255 - val_loss: 130.2676
Epoch 250/1000
1/1 [==============================] - 1s 1s/step - loss: 3.7639 - val_loss: 112.9722
Epoch 251/1000
1/1 [==============================] - 1s 1s/step - loss: 3.5317 - val_loss: 130.2495
Epoch 252/1000
1/1 [==============================] - 1s 1s/step - loss: 4.3513 - val_loss: 144.8465
Epoch 253/1000
1/1 [==============================] - 1s 1s/step - loss: 8.1201 - val_loss: 144.9306
Epoch 254/1000
1/1 [==============================] - 1s 1s/step - loss: 4.4336 - val_loss: 112.8192
Epoch

1/1 [==============================] - 2s 2s/step - loss: 3.7626 - val_loss: 143.6287
Epoch 327/1000
1/1 [==============================] - 1s 1s/step - loss: 3.6005 - val_loss: 112.3512
Epoch 328/1000
1/1 [==============================] - 1s 1s/step - loss: 3.0871 - val_loss: 112.4004
Epoch 329/1000
1/1 [==============================] - 1s 1s/step - loss: 4.4843 - val_loss: 112.5953
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 10.7052 - val_loss: 98.5164
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 4.1120 - val_loss: 112.5144
Epoch 332/1000
1/1 [==============================] - 1s 1s/step - loss: 4.1863 - val_loss: 112.6489
Epoch 333/1000
1/1 [==============================] - 1s 1s/step - loss: 3.0702 - val_loss: 143.7982
Epoch 334/1000
1/1 [==============================] - 1s 1s/step - loss: 3.5421 - val_loss: 129.6734
Epoch 335/1000
1/1 [==============================] - 1s 1s/step - loss: 4.0977 - val_loss: 129.4101
Epoch

...

1/1 [==============================] - 2s 2s/step - loss: 2.1832 - val_loss: 126.6495
Epoch 651/1000
1/1 [==============================] - 1s 1s/step - loss: 2.1161 - val_loss: 95.1146
Epoch 652/1000
1/1 [==============================] - 1s 1s/step - loss: 3.2068 - val_loss: 109.2679
Epoch 653/1000
1/1 [==============================] - 1s 1s/step - loss: 3.1652 - val_loss: 126.7965
Epoch 654/1000
1/1 [==============================] - 1s 1s/step - loss: 2.9523 - val_loss: 108.8896
Epoch 655/1000
1/1 [==============================] - 1s 1s/step - loss: 2.0511 - val_loss: 108.7899
Epoch 656/1000
1/1 [==============================] - 1s 1s/step - loss: 2.1583 - val_loss: 140.5352
Epoch 657/1000
1/1 [==============================] - 1s 1s/step - loss: 2.2010 - val_loss: 95.0910
Epoch 658/1000
1/1 [==============================] - 1s 1s/step - loss: 2.3470 - val_loss: 140.5034
Epoch 659/1000
1/1 [==============================] - 1s 1s/step - loss: 2.9105 - val_loss: 140.4613
Epoch 6

1/1 [==============================] - 1s 1s/step - loss: 2.8189 - val_loss: 108.3388
Epoch 732/1000
1/1 [==============================] - 2s 2s/step - loss: 4.3535 - val_loss: 122.6778
Epoch 733/1000
1/1 [==============================] - 1s 1s/step - loss: 2.9007 - val_loss: 140.5920
Epoch 734/1000
1/1 [==============================] - 2s 2s/step - loss: 2.1798 - val_loss: 126.6402
Epoch 735/1000
1/1 [==============================] - 1s 1s/step - loss: 4.7972 - val_loss: 108.8166
Epoch 736/1000
1/1 [==============================] - 1s 1s/step - loss: 4.1402 - val_loss: 140.6407
Epoch 737/1000
1/1 [==============================] - 1s 1s/step - loss: 2.3772 - val_loss: 140.4218
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 2.5221 - val_loss: 140.7259
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 2.1345 - val_loss: 140.4866
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 1.8652 - val_loss: 140.4189
Epoch

1/1 [==============================] - 1s 1s/step - loss: 1.7854 - val_loss: 138.9615
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 2.4796 - val_loss: 120.9444
Epoch 814/1000
1/1 [==============================] - 1s 1s/step - loss: 3.5164 - val_loss: 107.1147
Epoch 815/1000
1/1 [==============================] - 2s 2s/step - loss: 2.6050 - val_loss: 125.3016
Epoch 816/1000
1/1 [==============================] - 1s 1s/step - loss: 1.9269 - val_loss: 139.0816
Epoch 817/1000
1/1 [==============================] - 1s 1s/step - loss: 2.0702 - val_loss: 125.0857
Epoch 818/1000
1/1 [==============================] - 1s 1s/step - loss: 1.6139 - val_loss: 139.0762
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 1.8601 - val_loss: 125.2427
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 2.4436 - val_loss: 139.1373
Epoch 821/1000
1/1 [==============================] - 1s 1s/step - loss: 2.3122 - val_loss: 106.9005
Epoch

1/1 [==============================] - 1s 1s/step - loss: 4.8631 - val_loss: 138.8351
Epoch 894/1000
1/1 [==============================] - 1s 1s/step - loss: 1.8844 - val_loss: 92.8897
Epoch 895/1000
1/1 [==============================] - 1s 1s/step - loss: 1.9960 - val_loss: 125.0025
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6192 - val_loss: 106.8867
Epoch 897/1000
1/1 [==============================] - 1s 1s/step - loss: 1.5774 - val_loss: 124.9610
Epoch 898/1000
1/1 [==============================] - 1s 1s/step - loss: 4.2402 - val_loss: 125.1804
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9565 - val_loss: 107.1794
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6697 - val_loss: 107.0891
Epoch 901/1000
1/1 [==============================] - 1s 1s/step - loss: 1.7697 - val_loss: 139.0800
Epoch 902/1000
1/1 [==============================] - 1s 1s/step - loss: 1.6277 - val_loss: 107.0876
Epoch 

1/1 [==============================] - 1s 1s/step - loss: 1.5889 - val_loss: 124.7359
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9694 - val_loss: 106.7707
Epoch 976/1000
1/1 [==============================] - 2s 2s/step - loss: 2.1030 - val_loss: 138.4752
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4740 - val_loss: 106.5175
Epoch 978/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6183 - val_loss: 124.7871
Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6790 - val_loss: 124.7299
Epoch 980/1000
1/1 [==============================] - 1s 1s/step - loss: 1.4763 - val_loss: 106.2745
Epoch 981/1000
1/1 [==============================] - 1s 1s/step - loss: 2.8900 - val_loss: 106.5166
Epoch 982/1000
1/1 [==============================] - 1s 1s/step - loss: 2.9597 - val_loss: 106.8464
Epoch 983/1000
1/1 [==============================] - 1s 1s/step - loss: 1.9381 - val_loss: 106.8085
Epoch

In [12]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4754 - val_loss: 137.8245
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 3.0016 - val_loss: 124.0503
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 5.7536 - val_loss: 105.8612
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 1.8020 - val_loss: 138.0878
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 1.8358 - val_loss: 119.9331
Epoch 6/1000
1/1 [==============================] - 1s 1s/step - loss: 1.8121 - val_loss: 138.1745
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9706 - val_loss: 138.1992
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 2.4660 - val_loss: 137.9676
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6459 - val_loss: 124.1486
Epoch 10/1000
1/1 [==============================] - 1s 1s/step - loss: 2.1410 - val_loss: 137.8596
Epoch 11/

1/1 [==============================] - 2s 2s/step - loss: 1.5786 - val_loss: 137.4412
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6893 - val_loss: 123.6899
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 2.3090 - val_loss: 105.4126
Epoch 86/1000
1/1 [==============================] - 1s 1s/step - loss: 2.0653 - val_loss: 119.5898
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 17.2336 - val_loss: 123.2153
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 6.2827 - val_loss: 137.4162
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 1.5775 - val_loss: 105.7004
Epoch 90/1000
1/1 [==============================] - 1s 1s/step - loss: 1.4382 - val_loss: 105.7347
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4586 - val_loss: 105.6229
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9208 - val_loss: 91.6405
Epoch 93/1000


1/1 [==============================] - 2s 2s/step - loss: 3.2096 - val_loss: 91.1410
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4054 - val_loss: 137.0867
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 2.3080 - val_loss: 137.1012
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 1.5050 - val_loss: 123.8268
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 1.2712 - val_loss: 137.2753
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9885 - val_loss: 137.5872
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 1.5609 - val_loss: 104.9913
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 4.5340 - val_loss: 137.6047
Epoch 172/1000
1/1 [==============================] - 1s 1s/step - loss: 1.6918 - val_loss: 137.6990
Epoch 173/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1918 - val_loss: 137.6176
Epoch 

1/1 [==============================] - 2s 2s/step - loss: 1.4009 - val_loss: 136.0653
Epoch 246/1000
1/1 [==============================] - 1s 1s/step - loss: 1.0728 - val_loss: 104.4222
Epoch 247/1000
1/1 [==============================] - 1s 1s/step - loss: 1.5928 - val_loss: 122.2894
Epoch 248/1000
1/1 [==============================] - 1s 1s/step - loss: 1.8503 - val_loss: 136.4273
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 8.2086 - val_loss: 136.8750
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4219 - val_loss: 104.4902
Epoch 251/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1173 - val_loss: 118.2252
Epoch 252/1000
1/1 [==============================] - 1s 1s/step - loss: 1.2311 - val_loss: 136.6696
Epoch 253/1000
1/1 [==============================] - 1s 1s/step - loss: 1.8147 - val_loss: 122.9907
Epoch 254/1000
1/1 [==============================] - 1s 1s/step - loss: 1.5966 - val_loss: 104.4099
Epoch

1/1 [==============================] - 1s 1s/step - loss: 2.5280 - val_loss: 135.8018
Epoch 327/1000
1/1 [==============================] - 2s 2s/step - loss: 4.0307 - val_loss: 135.7755
Epoch 328/1000
1/1 [==============================] - 1s 1s/step - loss: 3.1175 - val_loss: 136.2113
Epoch 329/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1198 - val_loss: 89.6989
Epoch 330/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1576 - val_loss: 136.1385
Epoch 331/1000
1/1 [==============================] - 1s 1s/step - loss: 1.6706 - val_loss: 136.1176
Epoch 332/1000
1/1 [==============================] - 1s 1s/step - loss: 4.2481 - val_loss: 90.1081
Epoch 333/1000
1/1 [==============================] - 1s 1s/step - loss: 2.5868 - val_loss: 103.9303
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 1.0675 - val_loss: 122.6594
Epoch 335/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1414 - val_loss: 136.5393
Epoch 3

...

1/1 [==============================] - 1s 1s/step - loss: 0.9571 - val_loss: 101.5059
Epoch 651/1000
1/1 [==============================] - 1s 1s/step - loss: 1.4088 - val_loss: 121.2157
Epoch 652/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8461 - val_loss: 134.4553
Epoch 653/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8538 - val_loss: 101.8565
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9035 - val_loss: 120.8935
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 1.8497 - val_loss: 133.8106
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.9848 - val_loss: 153.3971
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6205 - val_loss: 120.8541
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 1.0068 - val_loss: 120.8213
Epoch 659/1000
1/1 [==============================] - 1s 1s/step - loss: 0.8705 - val_loss: 153.1452
Epoch

1/1 [==============================] - 1s 1s/step - loss: 0.7021 - val_loss: 101.7406
Epoch 732/1000
1/1 [==============================] - 1s 1s/step - loss: 1.3652 - val_loss: 134.0016
Epoch 733/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1500 - val_loss: 133.9375
Epoch 734/1000
1/1 [==============================] - 1s 1s/step - loss: 1.2363 - val_loss: 87.8735
Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 2.5897 - val_loss: 101.3035
Epoch 736/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7287 - val_loss: 120.4686
Epoch 737/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6699 - val_loss: 120.5093
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.9043 - val_loss: 133.7035
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4910 - val_loss: 120.5500
Epoch 740/1000
1/1 [==============================] - 1s 1s/step - loss: 1.3394 - val_loss: 134.0931
Epoch 

1/1 [==============================] - 2s 2s/step - loss: 1.4407 - val_loss: 87.9219
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9273 - val_loss: 120.3769
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6816 - val_loss: 101.2079
Epoch 815/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7409 - val_loss: 120.2429
Epoch 816/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8872 - val_loss: 101.0733
Epoch 817/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1462 - val_loss: 120.2766
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.9167 - val_loss: 100.7448
Epoch 819/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7956 - val_loss: 100.7758
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6842 - val_loss: 100.7369
Epoch 821/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7297 - val_loss: 100.6705
Epoch 

1/1 [==============================] - 2s 2s/step - loss: 0.9594 - val_loss: 99.7971
Epoch 894/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7564 - val_loss: 132.3282
Epoch 895/1000
1/1 [==============================] - 1s 1s/step - loss: 0.9742 - val_loss: 99.8396
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7067 - val_loss: 99.9412
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7380 - val_loss: 119.1720
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7241 - val_loss: 99.9962
Epoch 899/1000
1/1 [==============================] - 1s 1s/step - loss: 0.8259 - val_loss: 86.7177
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8694 - val_loss: 99.8056
Epoch 901/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7534 - val_loss: 119.2376
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6399 - val_loss: 119.1124
Epoch 903/1

1/1 [==============================] - 2s 2s/step - loss: 0.7250 - val_loss: 99.7060
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6397 - val_loss: 118.5499
Epoch 976/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5648 - val_loss: 118.6142
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 1.1406 - val_loss: 86.0755
Epoch 978/1000
1/1 [==============================] - 2s 2s/step - loss: 2.2522 - val_loss: 118.9494
Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6177 - val_loss: 132.6213
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6445 - val_loss: 118.9852
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6142 - val_loss: 132.6814
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6945 - val_loss: 119.2111
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7836 - val_loss: 119.2275
Epoch 9

In [13]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6213 - val_loss: 113.4743
Epoch 2/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5989 - val_loss: 99.6960
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7288 - val_loss: 99.8792
Epoch 4/1000
1/1 [==============================] - 1s 1s/step - loss: 1.5457 - val_loss: 85.9213
Epoch 5/1000
1/1 [==============================] - 1s 1s/step - loss: 0.8418 - val_loss: 118.7433
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 1.1744 - val_loss: 133.2727
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7912 - val_loss: 133.2056
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5941 - val_loss: 119.2416
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6233 - val_loss: 100.2602
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6773 - val_loss: 114.1289
Epoch 11/100

1/1 [==============================] - 1s 1s/step - loss: 2.0042 - val_loss: 132.2903
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 1.7643 - val_loss: 119.3008
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8710 - val_loss: 99.7387
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6423 - val_loss: 119.3370
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6725 - val_loss: 119.2248
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 1.2073 - val_loss: 99.6347
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 3.7714 - val_loss: 132.5831
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8379 - val_loss: 99.6747
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6899 - val_loss: 99.6230
Epoch 92/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6547 - val_loss: 118.9297
Epoch 93/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 0.5770 - val_loss: 131.4146
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6923 - val_loss: 99.0914
Epoch 166/1000
1/1 [==============================] - 1s 1s/step - loss: 1.3296 - val_loss: 112.1893
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 7.4751 - val_loss: 99.9629
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7359 - val_loss: 131.9088
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6498 - val_loss: 118.3242
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5509 - val_loss: 118.3779
Epoch 171/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6246 - val_loss: 118.3277
Epoch 172/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7173 - val_loss: 86.3026
Epoch 173/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7906 - val_loss: 99.6681
Epoch 174

1/1 [==============================] - 1s 1s/step - loss: 0.6054 - val_loss: 117.8258
Epoch 246/1000
1/1 [==============================] - 1s 1s/step - loss: 1.1518 - val_loss: 117.7466
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 1.1169 - val_loss: 99.0721
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6310 - val_loss: 131.2848
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7262 - val_loss: 118.0840
Epoch 250/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6058 - val_loss: 131.3856
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6855 - val_loss: 118.1691
Epoch 252/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6103 - val_loss: 117.9342
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5440 - val_loss: 99.0418
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5823 - val_loss: 98.8847
Epoch 25

1/1 [==============================] - 2s 2s/step - loss: 0.5023 - val_loss: 130.9378
Epoch 327/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5111 - val_loss: 117.5434
Epoch 328/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7948 - val_loss: 150.2950
Epoch 329/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7332 - val_loss: 130.8752
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 1.0266 - val_loss: 98.3977
Epoch 331/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5133 - val_loss: 98.4069
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5157 - val_loss: 130.9843
Epoch 333/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6526 - val_loss: 117.4226
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5597 - val_loss: 98.3983
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6153 - val_loss: 130.9819
Epoch 33

...

1/1 [==============================] - 1s 1s/step - loss: 0.5198 - val_loss: 116.5686
Epoch 651/1000
1/1 [==============================] - 2s 2s/step - loss: 0.9012 - val_loss: 96.9501
Epoch 652/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7773 - val_loss: 116.5400
Epoch 653/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5425 - val_loss: 116.6147
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4731 - val_loss: 97.2127
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3901 - val_loss: 129.4709
Epoch 656/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5306 - val_loss: 129.7150
Epoch 657/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4808 - val_loss: 116.6901
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4385 - val_loss: 116.7260
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4669 - val_loss: 116.7085
Epoch 6

1/1 [==============================] - 2s 2s/step - loss: 0.4381 - val_loss: 116.5035
Epoch 732/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4459 - val_loss: 96.4058
Epoch 733/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4648 - val_loss: 96.4060
Epoch 734/1000
1/1 [==============================] - 2s 2s/step - loss: 1.8443 - val_loss: 96.3080
Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5751 - val_loss: 129.7713
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4685 - val_loss: 149.7747
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5651 - val_loss: 96.6285
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3660 - val_loss: 96.6007
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6436 - val_loss: 109.9508
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 1.3303 - val_loss: 129.4886
Epoch 741/

1/1 [==============================] - 2s 2s/step - loss: 0.5933 - val_loss: 115.1717
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 2.8631 - val_loss: 96.6317
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 1.5773 - val_loss: 115.2556
Epoch 815/1000
1/1 [==============================] - 1s 1s/step - loss: 1.3088 - val_loss: 130.0629
Epoch 816/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3763 - val_loss: 115.7082
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4968 - val_loss: 129.6792
Epoch 818/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5730 - val_loss: 96.5941
Epoch 819/1000
1/1 [==============================] - 1s 1s/step - loss: 0.9228 - val_loss: 129.3655
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6170 - val_loss: 115.7510
Epoch 821/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3976 - val_loss: 115.6938
Epoch 8

1/1 [==============================] - 2s 2s/step - loss: 0.5257 - val_loss: 115.3444
Epoch 894/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3017 - val_loss: 95.9668
Epoch 895/1000
1/1 [==============================] - 2s 2s/step - loss: 0.6076 - val_loss: 128.3927
Epoch 896/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5346 - val_loss: 115.3945
Epoch 897/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4347 - val_loss: 115.3705
Epoch 898/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3686 - val_loss: 96.0255
Epoch 899/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4056 - val_loss: 95.8674
Epoch 900/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3257 - val_loss: 115.2338
Epoch 901/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6258 - val_loss: 95.7490
Epoch 902/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6961 - val_loss: 95.2294
Epoch 903/

1/1 [==============================] - 1s 1s/step - loss: 0.3709 - val_loss: 114.9693
Epoch 975/1000
1/1 [==============================] - 1s 1s/step - loss: 1.0552 - val_loss: 95.7977
Epoch 976/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4637 - val_loss: 127.8688
Epoch 977/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3290 - val_loss: 127.9254
Epoch 978/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5738 - val_loss: 95.7061
Epoch 979/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6685 - val_loss: 146.9681
Epoch 980/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4568 - val_loss: 114.7282
Epoch 981/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4692 - val_loss: 95.8438
Epoch 982/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3490 - val_loss: 127.9450
Epoch 983/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3845 - val_loss: 95.7120
Epoch 984

In [14]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8621 - val_loss: 114.0615
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8428 - val_loss: 95.5194
Epoch 3/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7065 - val_loss: 146.8623
Epoch 4/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3925 - val_loss: 114.7991
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3073 - val_loss: 114.8183
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3564 - val_loss: 82.5189
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3998 - val_loss: 114.7766
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4859 - val_loss: 127.7008
Epoch 9/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3680 - val_loss: 127.6578
Epoch 10/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3943 - val_loss: 114.8454
Epoch 11/10

1/1 [==============================] - 2s 2s/step - loss: 0.3654 - val_loss: 127.9797
Epoch 84/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4364 - val_loss: 82.6832
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4792 - val_loss: 114.9294
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4065 - val_loss: 96.0768
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5186 - val_loss: 128.2332
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3631 - val_loss: 114.8815
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3943 - val_loss: 128.3982
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5642 - val_loss: 95.5249
Epoch 91/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6712 - val_loss: 95.6964
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3554 - val_loss: 95.6346
Epoch 93/1000
1/1 [

1/1 [==============================] - 2s 2s/step - loss: 0.4870 - val_loss: 94.8081
Epoch 165/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5565 - val_loss: 94.9819
Epoch 166/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2695 - val_loss: 114.3616
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2949 - val_loss: 94.8969
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3927 - val_loss: 94.4973
Epoch 169/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4287 - val_loss: 114.2689
Epoch 170/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5563 - val_loss: 94.7056
Epoch 171/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3049 - val_loss: 114.5247
Epoch 172/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3205 - val_loss: 114.4541
Epoch 173/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4192 - val_loss: 127.6794
Epoch 174/

1/1 [==============================] - 1s 1s/step - loss: 0.3054 - val_loss: 114.6564
Epoch 246/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2763 - val_loss: 147.8458
Epoch 247/1000
1/1 [==============================] - 1s 1s/step - loss: 0.6509 - val_loss: 127.9263
Epoch 248/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4916 - val_loss: 114.7444
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2515 - val_loss: 114.8574
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3442 - val_loss: 115.0034
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 1.7005 - val_loss: 114.3388
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7527 - val_loss: 81.4132
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4321 - val_loss: 127.4620
Epoch 254/1000
1/1 [==============================] - 1s 1s/step - loss: 0.3973 - val_loss: 127.7893
Epoch 

1/1 [==============================] - 2s 2s/step - loss: 0.3545 - val_loss: 114.3179
Epoch 327/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2382 - val_loss: 128.0714
Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2410 - val_loss: 128.1117
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3639 - val_loss: 114.3804
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5705 - val_loss: 128.0624
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3366 - val_loss: 128.0744
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2762 - val_loss: 128.0154
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2934 - val_loss: 148.0317
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2594 - val_loss: 114.3796
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3785 - val_loss: 127.9749
Epoch

...

1/1 [==============================] - 2s 2s/step - loss: 0.2563 - val_loss: 113.2170
Epoch 651/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2320 - val_loss: 107.0197
Epoch 652/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2500 - val_loss: 126.8922
Epoch 653/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4422 - val_loss: 113.2938
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3604 - val_loss: 113.2408
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3664 - val_loss: 93.4891
Epoch 656/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2411 - val_loss: 93.5033
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2306 - val_loss: 93.4507
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2733 - val_loss: 93.2681
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4961 - val_loss: 93.7562
Epoch 660/

1/1 [==============================] - 2s 2s/step - loss: 0.3223 - val_loss: 112.3913
Epoch 732/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3504 - val_loss: 92.8875
Epoch 733/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2191 - val_loss: 92.8291
Epoch 734/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2254 - val_loss: 126.7016
Epoch 735/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2791 - val_loss: 145.9116
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2045 - val_loss: 92.8624
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.9479 - val_loss: 126.3996
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3271 - val_loss: 126.4514
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1725 - val_loss: 92.8388
Epoch 740/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1905 - val_loss: 92.8460
Epoch 741/

1/1 [==============================] - 2s 2s/step - loss: 0.1801 - val_loss: 91.8081
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2722 - val_loss: 78.5663
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2665 - val_loss: 125.1936
Epoch 815/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2302 - val_loss: 125.2863
Epoch 816/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2662 - val_loss: 78.3927
Epoch 817/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2234 - val_loss: 125.2308
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1886 - val_loss: 111.7991
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3314 - val_loss: 78.8334
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 4.6475 - val_loss: 92.8422
Epoch 821/1000
1/1 [==============================] - 1s 1s/step - loss: 0.4354 - val_loss: 92.9082
Epoch 822/1

1/1 [==============================] - 2s 2s/step - loss: 0.3654 - val_loss: 126.3091
Epoch 894/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3053 - val_loss: 106.9162
Epoch 895/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2588 - val_loss: 112.3940
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1781 - val_loss: 92.9898
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2034 - val_loss: 145.4867
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2103 - val_loss: 126.1378
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3501 - val_loss: 112.1817
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2115 - val_loss: 145.3213
Epoch 901/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2014 - val_loss: 92.6408
Epoch 902/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1827 - val_loss: 125.8185
Epoch 9

1/1 [==============================] - 2s 2s/step - loss: 0.1756 - val_loss: 125.5750
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2496 - val_loss: 92.3968
Epoch 976/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2714 - val_loss: 125.5432
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2236 - val_loss: 112.1155
Epoch 978/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2942 - val_loss: 125.8299
Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1733 - val_loss: 125.8343
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1786 - val_loss: 125.6831
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2538 - val_loss: 112.2030
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2236 - val_loss: 112.2162
Epoch 983/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2019 - val_loss: 125.6823
Epoch 

In [15]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1824 - val_loss: 124.9922
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2697 - val_loss: 91.7205
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2096 - val_loss: 111.1569
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2225 - val_loss: 91.8172
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2176 - val_loss: 125.0784
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1850 - val_loss: 91.9295
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1971 - val_loss: 92.0441
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2488 - val_loss: 91.7333
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2769 - val_loss: 111.1990
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3818 - val_loss: 91.5858
Epoch 11/1000
1

1/1 [==============================] - 2s 2s/step - loss: 0.2305 - val_loss: 105.5001
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1784 - val_loss: 91.7831
Epoch 85/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2317 - val_loss: 111.6801
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3412 - val_loss: 91.2238
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1982 - val_loss: 110.9985
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1845 - val_loss: 110.8981
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1822 - val_loss: 124.7325
Epoch 90/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2199 - val_loss: 124.7856
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1838 - val_loss: 91.3023
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1912 - val_loss: 144.7716
Epoch 93/1000
1/1

1/1 [==============================] - 2s 2s/step - loss: 1.5398 - val_loss: 125.2781
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2471 - val_loss: 112.0819
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9730 - val_loss: 91.8656
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2351 - val_loss: 125.9342
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1438 - val_loss: 112.9184
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2894 - val_loss: 125.8375
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3713 - val_loss: 112.4872
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3149 - val_loss: 91.7258
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2571 - val_loss: 91.8343
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1751 - val_loss: 112.2579
Epoch 17

1/1 [==============================] - 2s 2s/step - loss: 0.2221 - val_loss: 91.2504
Epoch 246/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1458 - val_loss: 91.2202
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1742 - val_loss: 125.1779
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1664 - val_loss: 111.2198
Epoch 249/1000
1/1 [==============================] - 1s 1s/step - loss: 4.3045 - val_loss: 91.0647
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.9742 - val_loss: 111.2848
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1924 - val_loss: 125.5225
Epoch 252/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1684 - val_loss: 90.9256
Epoch 253/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1571 - val_loss: 111.2870
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3309 - val_loss: 145.5621
Epoch 255

1/1 [==============================] - 2s 2s/step - loss: 0.1616 - val_loss: 105.3737
Epoch 327/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2232 - val_loss: 124.9564
Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2369 - val_loss: 91.4595
Epoch 329/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1558 - val_loss: 105.5906
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1624 - val_loss: 110.9885
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2091 - val_loss: 91.4012
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1593 - val_loss: 91.2179
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5196 - val_loss: 91.1340
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2516 - val_loss: 110.9919
Epoch 335/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1545 - val_loss: 105.7024
Epoch 336

...

1/1 [==============================] - 2s 2s/step - loss: 0.2933 - val_loss: 123.8198
Epoch 651/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2523 - val_loss: 124.2291
Epoch 652/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1272 - val_loss: 124.2263
Epoch 653/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1298 - val_loss: 124.2520
Epoch 654/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1644 - val_loss: 110.0678
Epoch 655/1000
1/1 [==============================] - 1s 1s/step - loss: 0.5540 - val_loss: 110.1645
Epoch 656/1000
1/1 [==============================] - 1s 1s/step - loss: 0.9262 - val_loss: 75.9889
Epoch 657/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1544 - val_loss: 124.6311
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1279 - val_loss: 124.5969
Epoch 659/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1579 - val_loss: 109.8340
Epoch 

1/1 [==============================] - 1s 1s/step - loss: 0.1364 - val_loss: 89.9184
Epoch 732/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2148 - val_loss: 124.4804
Epoch 733/1000
1/1 [==============================] - 2s 2s/step - loss: 0.9192 - val_loss: 124.3471
Epoch 734/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2302 - val_loss: 109.7009
Epoch 735/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2031 - val_loss: 109.7451
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1159 - val_loss: 124.1026
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1476 - val_loss: 90.0961
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2010 - val_loss: 109.7567
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4788 - val_loss: 109.2283
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1383 - val_loss: 123.8662
Epoch 7

1/1 [==============================] - 1s 1s/step - loss: 0.1862 - val_loss: 123.4511
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1373 - val_loss: 89.9224
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1300 - val_loss: 123.6163
Epoch 815/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1403 - val_loss: 90.1207
Epoch 816/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1230 - val_loss: 109.5632
Epoch 817/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1463 - val_loss: 109.5409
Epoch 818/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2462 - val_loss: 109.6114
Epoch 819/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2263 - val_loss: 124.1772
Epoch 820/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1448 - val_loss: 109.7779
Epoch 821/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1566 - val_loss: 76.0802
Epoch 82

1/1 [==============================] - 1s 1s/step - loss: 0.1192 - val_loss: 109.4053
Epoch 894/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2358 - val_loss: 90.0759
Epoch 895/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1647 - val_loss: 123.2006
Epoch 896/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2921 - val_loss: 123.1282
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1360 - val_loss: 90.3907
Epoch 898/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1245 - val_loss: 90.3490
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1417 - val_loss: 76.7068
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3106 - val_loss: 123.0472
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1775 - val_loss: 109.0183
Epoch 902/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1867 - val_loss: 90.0762
Epoch 903/

1/1 [==============================] - 1s 1s/step - loss: 0.2417 - val_loss: 122.5830
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1331 - val_loss: 122.5788
Epoch 976/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1263 - val_loss: 122.5902
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2210 - val_loss: 141.7268
Epoch 978/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1976 - val_loss: 89.3817
Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2236 - val_loss: 89.4655
Epoch 980/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1322 - val_loss: 89.4530
Epoch 981/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1452 - val_loss: 109.2526
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1660 - val_loss: 109.3064
Epoch 983/1000
1/1 [==============================] - 1s 1s/step - loss: 0.2288 - val_loss: 142.1317
Epoch 98

In [16]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1492 - val_loss: 108.4620
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1110 - val_loss: 90.0145
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1428 - val_loss: 108.5270
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5221 - val_loss: 108.5417
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2265 - val_loss: 89.6722
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4075 - val_loss: 122.3514
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1232 - val_loss: 89.6273
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1352 - val_loss: 89.7055
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1773 - val_loss: 108.5312
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5598 - val_loss: 123.2640
Epoch 11/1000

1/1 [==============================] - 2s 2s/step - loss: 0.5056 - val_loss: 90.4012
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1493 - val_loss: 122.3491
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1494 - val_loss: 108.5665
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1102 - val_loss: 90.0320
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1573 - val_loss: 122.4609
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1445 - val_loss: 108.5577
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1151 - val_loss: 122.4889
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1455 - val_loss: 122.4032
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1352 - val_loss: 108.6456
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1051 - val_loss: 122.6091
Epoch 93/1000
1/

1/1 [==============================] - 2s 2s/step - loss: 0.1068 - val_loss: 90.1138
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1160 - val_loss: 109.0029
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1278 - val_loss: 123.4538
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3799 - val_loss: 90.2221
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3953 - val_loss: 123.0061
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1559 - val_loss: 108.4499
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1020 - val_loss: 122.9300
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1083 - val_loss: 122.8214
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1694 - val_loss: 108.4934
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1291 - val_loss: 122.6391
Epoch 1

1/1 [==============================] - 2s 2s/step - loss: 0.1382 - val_loss: 122.3250
Epoch 246/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1627 - val_loss: 108.2498
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1409 - val_loss: 75.3641
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1287 - val_loss: 108.3299
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2291 - val_loss: 122.4570
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1830 - val_loss: 75.4053
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1701 - val_loss: 108.2057
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1838 - val_loss: 89.2491
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1006 - val_loss: 108.0388
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1351 - val_loss: 107.9487
Epoch 25

1/1 [==============================] - 2s 2s/step - loss: 0.0941 - val_loss: 122.4034
Epoch 327/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1039 - val_loss: 108.5272
Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1234 - val_loss: 89.6102
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1335 - val_loss: 122.3825
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1418 - val_loss: 89.2749
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1120 - val_loss: 141.3846
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1013 - val_loss: 122.3698
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1437 - val_loss: 108.4400
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1373 - val_loss: 108.4401
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1334 - val_loss: 89.4765
Epoch 33

...

1/1 [==============================] - 2s 2s/step - loss: 0.0998 - val_loss: 88.6580
Epoch 651/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1909 - val_loss: 108.1251
Epoch 652/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2318 - val_loss: 121.8890
Epoch 653/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1393 - val_loss: 88.5421
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0880 - val_loss: 121.8122
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0831 - val_loss: 74.6388
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1005 - val_loss: 107.7320
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1050 - val_loss: 121.9288
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1168 - val_loss: 88.3634
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1300 - val_loss: 121.5541
Epoch 660

1/1 [==============================] - 2s 2s/step - loss: 0.1988 - val_loss: 120.2750
Epoch 732/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1138 - val_loss: 106.3786
Epoch 733/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0841 - val_loss: 88.0272
Epoch 734/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1230 - val_loss: 106.5931
Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1096 - val_loss: 88.1455
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1175 - val_loss: 106.7784
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0928 - val_loss: 120.9389
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1060 - val_loss: 88.3734
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1096 - val_loss: 102.6038
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2036 - val_loss: 88.6440
Epoch 741

1/1 [==============================] - 2s 2s/step - loss: 0.1032 - val_loss: 87.5467
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0857 - val_loss: 87.5810
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1120 - val_loss: 120.9179
Epoch 815/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0966 - val_loss: 106.5708
Epoch 816/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0805 - val_loss: 106.5673
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0870 - val_loss: 106.5638
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1229 - val_loss: 120.6597
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1629 - val_loss: 87.9951
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1362 - val_loss: 87.7407
Epoch 821/1000
1/1 [==============================] - 1s 1s/step - loss: 0.1241 - val_loss: 106.2597
Epoch 822

1/1 [==============================] - 2s 2s/step - loss: 0.0971 - val_loss: 120.4698
Epoch 894/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0845 - val_loss: 88.4646
Epoch 895/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0982 - val_loss: 120.6015
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1247 - val_loss: 88.4241
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1269 - val_loss: 106.5573
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1012 - val_loss: 102.2191
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0885 - val_loss: 106.5805
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0751 - val_loss: 106.6157
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0998 - val_loss: 120.6024
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0962 - val_loss: 120.5395
Epoch 9

1/1 [==============================] - 2s 2s/step - loss: 0.1047 - val_loss: 120.1673
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1039 - val_loss: 105.7477
Epoch 976/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1013 - val_loss: 105.7179
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0778 - val_loss: 120.0242
Epoch 978/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0790 - val_loss: 87.9113
Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1548 - val_loss: 87.7089
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1557 - val_loss: 119.8948
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1054 - val_loss: 105.7977
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1143 - val_loss: 87.9950
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0930 - val_loss: 88.0595
Epoch 984

In [17]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1609 - val_loss: 105.9369
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1271 - val_loss: 120.0410
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0830 - val_loss: 105.5437
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0831 - val_loss: 120.4111
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1110 - val_loss: 105.6464
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1129 - val_loss: 105.7588
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1112 - val_loss: 87.9599
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1106 - val_loss: 105.7379
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0898 - val_loss: 88.4047
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0942 - val_loss: 120.6879
Epoch 11/10

1/1 [==============================] - 2s 2s/step - loss: 0.0785 - val_loss: 87.5103
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1798 - val_loss: 101.9617
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1468 - val_loss: 87.4036
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1011 - val_loss: 105.5806
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0921 - val_loss: 138.2680
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0793 - val_loss: 87.5119
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0825 - val_loss: 105.7081
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0900 - val_loss: 101.8504
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 1.3104 - val_loss: 106.4507
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2268 - val_loss: 106.4701
Epoch 93/1000
1/1

1/1 [==============================] - 2s 2s/step - loss: 0.1527 - val_loss: 87.8699
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1079 - val_loss: 103.2913
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0845 - val_loss: 87.8946
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2152 - val_loss: 120.1566
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0905 - val_loss: 120.1512
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2583 - val_loss: 120.1539
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1923 - val_loss: 104.9186
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1085 - val_loss: 137.2762
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0783 - val_loss: 104.9141
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1084 - val_loss: 104.9127
Epoch 1

1/1 [==============================] - 2s 2s/step - loss: 0.0937 - val_loss: 119.5419
Epoch 246/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1101 - val_loss: 102.1661
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0975 - val_loss: 104.8852
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0760 - val_loss: 104.7927
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0765 - val_loss: 119.5198
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0780 - val_loss: 87.3180
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0739 - val_loss: 87.2970
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0950 - val_loss: 104.7112
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1299 - val_loss: 119.5645
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0973 - val_loss: 104.8141
Epoch 2

1/1 [==============================] - 2s 2s/step - loss: 0.1513 - val_loss: 105.3562
Epoch 327/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0864 - val_loss: 137.6344
Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1008 - val_loss: 87.1289
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0735 - val_loss: 87.1811
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1469 - val_loss: 119.3874
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 1.5112 - val_loss: 87.5075
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0997 - val_loss: 87.5607
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0995 - val_loss: 87.5658
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1297 - val_loss: 87.5658
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 2.8712 - val_loss: 87.8635
Epoch 336/10

...

1/1 [==============================] - 2s 2s/step - loss: 0.0812 - val_loss: 104.8785
Epoch 651/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0750 - val_loss: 119.3552
Epoch 652/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1181 - val_loss: 119.2840
Epoch 653/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1597 - val_loss: 118.9658
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1895 - val_loss: 137.1902
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0943 - val_loss: 86.8956
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0676 - val_loss: 119.2353
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0718 - val_loss: 86.8846
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0914 - val_loss: 119.1656
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0907 - val_loss: 119.1463
Epoch 6

1/1 [==============================] - 2s 2s/step - loss: 0.1204 - val_loss: 120.0489
Epoch 732/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0879 - val_loss: 119.9476
Epoch 733/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0737 - val_loss: 88.1227
Epoch 734/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0643 - val_loss: 88.0928
Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0804 - val_loss: 136.7441
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1373 - val_loss: 87.9052
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2103 - val_loss: 119.2214
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1049 - val_loss: 87.7265
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0674 - val_loss: 104.6650
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0724 - val_loss: 87.6394
Epoch 741/

1/1 [==============================] - 2s 2s/step - loss: 0.0635 - val_loss: 104.4227
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0679 - val_loss: 72.2986
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0795 - val_loss: 119.3364
Epoch 815/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7332 - val_loss: 87.0438
Epoch 816/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2077 - val_loss: 119.2023
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1029 - val_loss: 119.0666
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0650 - val_loss: 119.1003
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0738 - val_loss: 102.0231
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0635 - val_loss: 87.0269
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1475 - val_loss: 104.1303
Epoch 82

1/1 [==============================] - 2s 2s/step - loss: 0.0936 - val_loss: 103.8348
Epoch 894/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0914 - val_loss: 103.9448
Epoch 895/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0615 - val_loss: 118.4538
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1600 - val_loss: 86.3824
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1234 - val_loss: 103.7020
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1173 - val_loss: 103.6597
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0936 - val_loss: 118.2232
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0790 - val_loss: 100.6747
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0654 - val_loss: 118.2099
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0637 - val_loss: 118.3167
Epoch 

1/1 [==============================] - 2s 2s/step - loss: 0.0580 - val_loss: 104.5243
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0918 - val_loss: 136.5509
Epoch 976/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1056 - val_loss: 118.6134
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0887 - val_loss: 86.4138
Epoch 978/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0831 - val_loss: 136.7021
Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0620 - val_loss: 118.6649
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0679 - val_loss: 118.5312
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0871 - val_loss: 72.4738
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2369 - val_loss: 86.4564
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0981 - val_loss: 104.3661
Epoch 98

In [18]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0628 - val_loss: 86.6911
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0765 - val_loss: 101.4207
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0841 - val_loss: 104.6344
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0674 - val_loss: 119.0363
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0917 - val_loss: 86.8234
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0791 - val_loss: 86.8900
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1111 - val_loss: 104.4527
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1428 - val_loss: 104.4629
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1766 - val_loss: 118.9056
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0809 - val_loss: 118.7290
Epoch 11/100

1/1 [==============================] - 2s 2s/step - loss: 0.0890 - val_loss: 136.0600
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0565 - val_loss: 86.5453
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0706 - val_loss: 103.8574
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0671 - val_loss: 71.4320
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0748 - val_loss: 103.7624
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1143 - val_loss: 103.7161
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0622 - val_loss: 103.6961
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0648 - val_loss: 86.4004
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0796 - val_loss: 86.3651
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1124 - val_loss: 103.6949
Epoch 93/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 0.0962 - val_loss: 70.7157
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0805 - val_loss: 85.2323
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0698 - val_loss: 85.2606
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0852 - val_loss: 85.2523
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0665 - val_loss: 85.2606
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0799 - val_loss: 85.2559
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1179 - val_loss: 85.1893
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1278 - val_loss: 85.1656
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.4492 - val_loss: 103.4633
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0613 - val_loss: 117.9815
Epoch 174/100

1/1 [==============================] - 2s 2s/step - loss: 0.0631 - val_loss: 135.8259
Epoch 246/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0606 - val_loss: 85.4798
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0691 - val_loss: 117.9777
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1338 - val_loss: 118.0213
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1077 - val_loss: 103.0220
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1032 - val_loss: 117.7744
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0610 - val_loss: 85.3429
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0610 - val_loss: 103.0415
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0990 - val_loss: 117.7888
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1036 - val_loss: 99.9761
Epoch 25

1/1 [==============================] - 2s 2s/step - loss: 0.1270 - val_loss: 118.4115
Epoch 327/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0649 - val_loss: 85.9297
Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0581 - val_loss: 103.2810
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0630 - val_loss: 103.3606
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0601 - val_loss: 85.8304
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0819 - val_loss: 85.8688
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0817 - val_loss: 117.8740
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0709 - val_loss: 85.8826
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0638 - val_loss: 85.9565
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0608 - val_loss: 135.0864
Epoch 336/

...

1/1 [==============================] - 2s 2s/step - loss: 0.0693 - val_loss: 85.6430
Epoch 651/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1158 - val_loss: 85.5433
Epoch 652/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0524 - val_loss: 85.6030
Epoch 653/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0538 - val_loss: 100.1716
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0899 - val_loss: 103.2600
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0873 - val_loss: 103.1011
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0813 - val_loss: 85.6812
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0657 - val_loss: 85.6522
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0605 - val_loss: 85.6140
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0621 - val_loss: 85.5450
Epoch 660/10

1/1 [==============================] - 2s 2s/step - loss: 0.0558 - val_loss: 99.5977
Epoch 732/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0675 - val_loss: 116.7890
Epoch 733/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3057 - val_loss: 118.2389
Epoch 734/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5013 - val_loss: 84.9352
Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1139 - val_loss: 102.3855
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0693 - val_loss: 85.1406
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0711 - val_loss: 70.6469
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1107 - val_loss: 102.2813
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0833 - val_loss: 102.2326
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1353 - val_loss: 102.6329
Epoch 741

1/1 [==============================] - 2s 2s/step - loss: 0.0737 - val_loss: 70.3940
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0871 - val_loss: 102.6371
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0467 - val_loss: 117.3368
Epoch 815/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0987 - val_loss: 117.3367
Epoch 816/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0797 - val_loss: 84.9573
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0808 - val_loss: 116.8161
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0559 - val_loss: 134.1735
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0525 - val_loss: 116.9130
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0502 - val_loss: 85.0457
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0815 - val_loss: 117.0947
Epoch 82

1/1 [==============================] - 2s 2s/step - loss: 0.2092 - val_loss: 99.3634
Epoch 894/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0601 - val_loss: 116.6198
Epoch 895/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0613 - val_loss: 116.6568
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0705 - val_loss: 116.6477
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0538 - val_loss: 116.6635
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0511 - val_loss: 102.2606
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0462 - val_loss: 99.4977
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0705 - val_loss: 116.8574
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0878 - val_loss: 116.9777
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0549 - val_loss: 70.5296
Epoch 90

1/1 [==============================] - 2s 2s/step - loss: 0.0600 - val_loss: 133.4922
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0557 - val_loss: 84.5766
Epoch 976/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0513 - val_loss: 116.0711
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0530 - val_loss: 102.0768
Epoch 978/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0955 - val_loss: 84.4035
Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0582 - val_loss: 101.7967
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0629 - val_loss: 84.5097
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0455 - val_loss: 101.8710
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0514 - val_loss: 116.2024
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1635 - val_loss: 101.8863
Epoch 98

In [19]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0783 - val_loss: 84.8678
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0477 - val_loss: 116.4831
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0464 - val_loss: 102.0297
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1026 - val_loss: 84.9662
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8286 - val_loss: 84.5750
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1180 - val_loss: 116.2789
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0676 - val_loss: 70.0656
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0710 - val_loss: 116.3832
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0534 - val_loss: 116.2830
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0915 - val_loss: 101.9593
Epoch 11/1000

1/1 [==============================] - 1s 1s/step - loss: 0.3692 - val_loss: 101.9312
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0876 - val_loss: 102.0460
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0937 - val_loss: 116.5173
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0496 - val_loss: 84.2481
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0482 - val_loss: 102.0214
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0565 - val_loss: 101.9803
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1108 - val_loss: 84.1942
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1026 - val_loss: 116.6960
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0442 - val_loss: 102.1053
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0497 - val_loss: 84.2899
Epoch 93/1000
1/1

1/1 [==============================] - 2s 2s/step - loss: 0.0532 - val_loss: 101.6603
Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0620 - val_loss: 116.0892
Epoch 166/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0727 - val_loss: 84.2194
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0637 - val_loss: 84.3819
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0530 - val_loss: 116.1411
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0433 - val_loss: 116.1219
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0438 - val_loss: 84.3954
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0470 - val_loss: 84.4260
Epoch 172/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0608 - val_loss: 84.5214
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0838 - val_loss: 84.4542
Epoch 174/1

1/1 [==============================] - 2s 2s/step - loss: 0.0719 - val_loss: 84.6923
Epoch 246/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0510 - val_loss: 116.6838
Epoch 247/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0426 - val_loss: 116.6889
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0453 - val_loss: 116.7363
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0508 - val_loss: 116.7200
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0859 - val_loss: 116.4464
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0730 - val_loss: 84.3577
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0403 - val_loss: 116.3554
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0445 - val_loss: 84.3861
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0505 - val_loss: 116.4314
Epoch 25

1/1 [==============================] - 2s 2s/step - loss: 0.0490 - val_loss: 116.8954
Epoch 327/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0595 - val_loss: 84.2838
Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0578 - val_loss: 98.9166
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0737 - val_loss: 84.0241
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0569 - val_loss: 116.5236
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0460 - val_loss: 116.5450
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0404 - val_loss: 116.6063
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0854 - val_loss: 116.5364
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0861 - val_loss: 116.4257
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0752 - val_loss: 116.3257
Epoch 33

...

1/1 [==============================] - 1s 1s/step - loss: 0.0415 - val_loss: 83.9766
Epoch 651/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0418 - val_loss: 101.2580
Epoch 652/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0578 - val_loss: 98.9043
Epoch 653/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0564 - val_loss: 115.6764
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0373 - val_loss: 83.9225
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0457 - val_loss: 116.0080
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0494 - val_loss: 84.1104
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0623 - val_loss: 100.9793
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0710 - val_loss: 115.8273
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0477 - val_loss: 115.9199
Epoch 660

1/1 [==============================] - 2s 2s/step - loss: 0.0576 - val_loss: 115.7223
Epoch 732/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0390 - val_loss: 101.0790
Epoch 733/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0420 - val_loss: 84.1090
Epoch 734/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0492 - val_loss: 84.1508
Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0468 - val_loss: 100.9658
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0728 - val_loss: 99.0508
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0372 - val_loss: 115.6850
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0421 - val_loss: 100.9429
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0474 - val_loss: 131.9783
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0430 - val_loss: 100.6214
Epoch 74

1/1 [==============================] - 1s 1s/step - loss: 0.0952 - val_loss: 83.0656
Epoch 813/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0465 - val_loss: 83.2013
Epoch 814/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0435 - val_loss: 68.1875
Epoch 815/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0398 - val_loss: 100.3738
Epoch 816/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0387 - val_loss: 100.4119
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0438 - val_loss: 100.4408
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0583 - val_loss: 83.2631
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1501 - val_loss: 100.1006
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0439 - val_loss: 132.3087
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0492 - val_loss: 83.2299
Epoch 822/

1/1 [==============================] - 2s 2s/step - loss: 0.0457 - val_loss: 83.3071
Epoch 894/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0420 - val_loss: 83.4188
Epoch 895/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0379 - val_loss: 100.1839
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0434 - val_loss: 100.2186
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0546 - val_loss: 83.4632
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0411 - val_loss: 114.9758
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0421 - val_loss: 114.9594
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0373 - val_loss: 100.1612
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0430 - val_loss: 83.7066
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0422 - val_loss: 83.7669
Epoch 903/

1/1 [==============================] - 2s 2s/step - loss: 0.0481 - val_loss: 83.3155
Epoch 975/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0532 - val_loss: 114.9333
Epoch 976/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0542 - val_loss: 83.3797
Epoch 977/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0362 - val_loss: 100.1115
Epoch 978/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0345 - val_loss: 100.2139
Epoch 979/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0398 - val_loss: 115.0146
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0914 - val_loss: 83.6637
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0593 - val_loss: 83.3652
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0460 - val_loss: 98.1423
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0367 - val_loss: 83.4635
Epoch 984/1

In [20]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0638 - val_loss: 114.3746
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0623 - val_loss: 99.9644
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0552 - val_loss: 83.2565
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0390 - val_loss: 99.9904
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0332 - val_loss: 100.0320
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0451 - val_loss: 100.0557
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0450 - val_loss: 114.7375
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0922 - val_loss: 83.2346
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0572 - val_loss: 83.1783
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0335 - val_loss: 114.4497
Epoch 11/1000


1/1 [==============================] - 2s 2s/step - loss: 0.0376 - val_loss: 83.3308
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0394 - val_loss: 131.1657
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0430 - val_loss: 114.5886
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0521 - val_loss: 99.9769
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0327 - val_loss: 114.5534
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0396 - val_loss: 83.1706
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0395 - val_loss: 114.6398
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0390 - val_loss: 114.7080
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0403 - val_loss: 83.2892
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0523 - val_loss: 114.7187
Epoch 93/1000
1/1 

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0394 - val_loss: 83.4130
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0350 - val_loss: 114.5146
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0383 - val_loss: 100.0925
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0700 - val_loss: 130.8688
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0784 - val_loss: 99.7113
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0400 - val_loss: 99.7623
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0333 - val_loss: 97.2818
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0329 - val_loss: 114.1287
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0365 - val_loss: 114.1695
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0473 - val_loss: 114

1/1 [==============================] - 2s 2s/step - loss: 0.0727 - val_loss: 115.6127
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0437 - val_loss: 115.5536
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0593 - val_loss: 115.7606
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0372 - val_loss: 132.5546
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0394 - val_loss: 83.2648
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0583 - val_loss: 83.1180
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0437 - val_loss: 98.5978
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0400 - val_loss: 100.0173
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0408 - val_loss: 100.0378
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0331 - val_loss: 100.0258
Epoch 25

1/1 [==============================] - 2s 2s/step - loss: 0.0346 - val_loss: 99.3927
Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0354 - val_loss: 82.7818
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0624 - val_loss: 99.4924
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0577 - val_loss: 82.8349
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0688 - val_loss: 114.2533
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0342 - val_loss: 99.4950
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0360 - val_loss: 130.8387
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0357 - val_loss: 99.5031
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0452 - val_loss: 99.5657
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0445 - val_loss: 82.9792
Epoch 337/100

...

1/1 [==============================] - 2s 2s/step - loss: 0.0383 - val_loss: 113.2796
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0527 - val_loss: 67.8731
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0292 - val_loss: 113.5138
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0314 - val_loss: 98.8211
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0286 - val_loss: 113.3933
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0376 - val_loss: 113.3980
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0409 - val_loss: 82.3743
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0653 - val_loss: 129.5268
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0367 - val_loss: 82.5972
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0474 - val_loss: 98.9183
Epoch 663/

Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0427 - val_loss: 129.6920
Epoch 736/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0688 - val_loss: 113.2250
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0400 - val_loss: 113.2385
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0345 - val_loss: 68.2638
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0345 - val_loss: 82.4577
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0298 - val_loss: 113.2218
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0357 - val_loss: 82.4705
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0370 - val_loss: 82.4644
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0373 - val_loss: 82.4752
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0360 - val_loss: 82.3

1/1 [==============================] - 2s 2s/step - loss: 0.0321 - val_loss: 82.3233
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0682 - val_loss: 98.7643
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0284 - val_loss: 113.3211
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0677 - val_loss: 129.9887
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0741 - val_loss: 99.0230
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0371 - val_loss: 99.0546
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0323 - val_loss: 82.1881
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0302 - val_loss: 98.9481
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0339 - val_loss: 98.8565
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0582 - val_loss: 113.3977
Epoch 826/10

Epoch 898/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0368 - val_loss: 81.3710
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0353 - val_loss: 113.2113
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0403 - val_loss: 129.9809
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0278 - val_loss: 113.1371
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0291 - val_loss: 113.1289
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0480 - val_loss: 112.9929
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1417 - val_loss: 112.6288
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0409 - val_loss: 112.5582
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0307 - val_loss: 128.9047
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0286 - val_loss: 

1/1 [==============================] - 2s 2s/step - loss: 0.0323 - val_loss: 82.2928
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0327 - val_loss: 98.2514
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1013 - val_loss: 82.5975
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0690 - val_loss: 82.2272
Epoch 983/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0765 - val_loss: 82.4435
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0388 - val_loss: 113.0976
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0293 - val_loss: 113.1000
Epoch 986/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0438 - val_loss: 113.2993
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0782 - val_loss: 113.1470
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0534 - val_loss: 82.0575
Epoch 989/1

In [21]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0310 - val_loss: 98.0366
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0708 - val_loss: 112.6218
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0454 - val_loss: 81.6735
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0382 - val_loss: 113.0476
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0343 - val_loss: 81.8178
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0270 - val_loss: 81.8435
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0310 - val_loss: 113.1153
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0408 - val_loss: 129.4869
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0498 - val_loss: 82.0966
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0444 - val_loss: 113.0284
Epoch 11/1000


1/1 [==============================] - 2s 2s/step - loss: 0.0331 - val_loss: 81.5335
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0265 - val_loss: 98.5323
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0297 - val_loss: 113.2926
Epoch 86/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0350 - val_loss: 66.7565
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0478 - val_loss: 98.2388
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1543 - val_loss: 81.8424
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0491 - val_loss: 98.2977
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - val_loss: 81.8350
Epoch 91/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0295 - val_loss: 81.8598
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0429 - val_loss: 96.6201
Epoch 93/1000
1/1 [====

1/1 [==============================] - 2s 2s/step - loss: 0.0508 - val_loss: 81.7396
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0548 - val_loss: 113.4836
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0415 - val_loss: 96.6024
Epoch 168/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0372 - val_loss: 81.6865
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0437 - val_loss: 113.4921
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0309 - val_loss: 113.4304
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0425 - val_loss: 113.4189
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0417 - val_loss: 113.6178
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0321 - val_loss: 98.5191
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0311 - val_loss: 98.5755
Epoch 175/

Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0555 - val_loss: 97.9617
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0861 - val_loss: 81.4756
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0524 - val_loss: 81.3569
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0335 - val_loss: 97.7277
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0283 - val_loss: 81.5169
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0338 - val_loss: 81.5852
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0294 - val_loss: 81.5447
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0603 - val_loss: 66.3880
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0419 - val_loss: 113.2734
Epoch 256/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0387 - val_loss: 98.1693

1/1 [==============================] - 2s 2s/step - loss: 0.0832 - val_loss: 113.0399
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0271 - val_loss: 97.2631
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0260 - val_loss: 97.9456
Epoch 331/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0394 - val_loss: 98.1033
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0463 - val_loss: 113.1079
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1897 - val_loss: 97.6656
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0323 - val_loss: 97.6919
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0258 - val_loss: 128.2406
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0282 - val_loss: 82.0224
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7545 - val_loss: 111.6145
Epoch 338/1

...

1/1 [==============================] - 2s 2s/step - loss: 0.0386 - val_loss: 112.6669
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0645 - val_loss: 97.6853
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0368 - val_loss: 81.7779
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0323 - val_loss: 97.6000
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0254 - val_loss: 81.8131
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 97.5623
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0426 - val_loss: 81.7710
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0547 - val_loss: 81.6879
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0403 - val_loss: 112.5574
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0262 - val_loss: 97.3208
Epoch 664/100

Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0262 - val_loss: 80.7723
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0254 - val_loss: 111.5070
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0264 - val_loss: 111.4381
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0284 - val_loss: 111.4283
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0526 - val_loss: 66.5210
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0394 - val_loss: 111.4848
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0265 - val_loss: 81.1919
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0344 - val_loss: 95.5982
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - val_loss: 96.9557
Epoch 745/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0377 - val_loss: 111.

1/1 [==============================] - 2s 2s/step - loss: 0.1763 - val_loss: 79.9526
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0316 - val_loss: 97.0219
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0260 - val_loss: 97.0552
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 80.2463
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0320 - val_loss: 97.0318
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0384 - val_loss: 80.3986
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0271 - val_loss: 111.6857
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0310 - val_loss: 65.9784
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0295 - val_loss: 97.2571
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 97.1806
Epoch 827/1000

Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0316 - val_loss: 127.9391
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0267 - val_loss: 80.7317
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0587 - val_loss: 111.8609
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0347 - val_loss: 111.8258
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0270 - val_loss: 111.8140
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0258 - val_loss: 80.7647
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0307 - val_loss: 97.1521
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0475 - val_loss: 111.7031
Epoch 907/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0270 - val_loss: 80.7991
Epoch 908/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 66.

1/1 [==============================] - 2s 2s/step - loss: 0.0284 - val_loss: 81.1919
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0285 - val_loss: 97.0997
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0266 - val_loss: 111.5208
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0325 - val_loss: 81.1801
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0334 - val_loss: 97.1105
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0326 - val_loss: 111.5810
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0275 - val_loss: 81.1667
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0239 - val_loss: 81.1445
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0265 - val_loss: 81.1759
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0330 - val_loss: 81.1421
Epoch 990/100

In [22]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0621 - val_loss: 97.5776
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0286 - val_loss: 97.5058
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0221 - val_loss: 112.1467
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0245 - val_loss: 66.6076
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0680 - val_loss: 81.1167
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0580 - val_loss: 97.3527
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0396 - val_loss: 95.8770
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 81.1069
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0280 - val_loss: 81.0703
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0295 - val_loss: 111.9482
Epoch 11/1000
1/1

1/1 [==============================] - 2s 2s/step - loss: 0.0447 - val_loss: 96.8077
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0319 - val_loss: 96.8476
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - val_loss: 96.7383
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0276 - val_loss: 80.7964
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0309 - val_loss: 96.7641
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0366 - val_loss: 95.4892
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0510 - val_loss: 111.7652
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0317 - val_loss: 96.9038
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0289 - val_loss: 80.8284
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0291 - val_loss: 111.4815
Epoch 93/1000
1/1 [===

1/1 [==============================] - 2s 2s/step - loss: 0.0306 - val_loss: 111.3510
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0489 - val_loss: 111.3993
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0383 - val_loss: 111.4716
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0260 - val_loss: 97.0150
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0224 - val_loss: 111.4392
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 96.9719
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0398 - val_loss: 97.0521
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0801 - val_loss: 80.7779
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0270 - val_loss: 112.2352
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0292 - val_loss: 80.7044
Epoch 175/

1/1 [==============================] - 2s 2s/step - loss: 0.0275 - val_loss: 111.2035
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0246 - val_loss: 80.9059
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0255 - val_loss: 96.6909
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0391 - val_loss: 111.1016
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - val_loss: 95.3303
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0801 - val_loss: 81.2077
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0251 - val_loss: 81.2360
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0229 - val_loss: 96.6648
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 96.6022
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0707 - val_loss: 96.7248
Epoch 256/100

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0306 - val_loss: 96.6724
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0309 - val_loss: 80.6413
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0265 - val_loss: 111.1661
Epoch 331/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0234 - val_loss: 95.4853
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0239 - val_loss: 96.4509
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0276 - val_loss: 96.5350
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0305 - val_loss: 80.7314
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0374 - val_loss: 111.3905
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0299 - val_loss: 96.7294
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 111.49

...

1/1 [==============================] - 2s 2s/step - loss: 0.0238 - val_loss: 95.9933
Epoch 654/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0219 - val_loss: 80.3734
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0298 - val_loss: 126.8750
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0328 - val_loss: 96.1483
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0412 - val_loss: 80.5428
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0224 - val_loss: 80.5593
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0248 - val_loss: 96.2451
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0231 - val_loss: 80.4819
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 96.1874
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0282 - val_loss: 111.2759
Epoch 663/100

Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0499 - val_loss: 95.8325
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0571 - val_loss: 126.0634
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0256 - val_loss: 95.7734
Epoch 738/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0208 - val_loss: 79.9212
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0211 - val_loss: 79.9090
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0361 - val_loss: 79.7637
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0339 - val_loss: 95.5651
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0293 - val_loss: 109.9644
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0218 - val_loss: 79.8837
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0257 - val_loss: 110.06

1/1 [==============================] - 2s 2s/step - loss: 0.0260 - val_loss: 80.3311
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 80.3902
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0229 - val_loss: 80.3061
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0256 - val_loss: 110.7670
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0227 - val_loss: 126.9285
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0212 - val_loss: 110.7310
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0194 - val_loss: 96.4603
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0238 - val_loss: 110.7090
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0339 - val_loss: 80.3629
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0258 - val_loss: 80.3493
Epoch 826/1

Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0226 - val_loss: 95.4532
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0207 - val_loss: 79.7132
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0223 - val_loss: 109.8156
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0274 - val_loss: 79.8232
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0289 - val_loss: 95.6840
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0331 - val_loss: 110.0947
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0275 - val_loss: 110.0810
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0211 - val_loss: 95.7709
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0285 - val_loss: 110.1644
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0288 - val_loss: 110.

1/1 [==============================] - 2s 2s/step - loss: 0.0239 - val_loss: 79.9615
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 95.1429
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0272 - val_loss: 109.8504
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 95.2987
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0226 - val_loss: 79.8274
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0424 - val_loss: 79.9543
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0822 - val_loss: 109.2381
Epoch 986/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0407 - val_loss: 95.0418
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0269 - val_loss: 95.1532
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0206 - val_loss: 95.1351
Epoch 989/100

In [23]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0444 - val_loss: 95.4494
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0478 - val_loss: 109.4985
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0240 - val_loss: 95.3484
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0254 - val_loss: 109.5091
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0296 - val_loss: 109.6801
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0234 - val_loss: 125.3711
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0470 - val_loss: 79.5566
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0305 - val_loss: 79.6456
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0211 - val_loss: 65.2589
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0243 - val_loss: 79.7254
Epoch 11/1000
1

1/1 [==============================] - 2s 2s/step - loss: 0.0222 - val_loss: 79.5476
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0302 - val_loss: 96.2532
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0250 - val_loss: 96.1550
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0222 - val_loss: 110.4739
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0273 - val_loss: 96.1819
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0200 - val_loss: 96.0785
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 79.6588
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0281 - val_loss: 126.8039
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0553 - val_loss: 79.8497
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0283 - val_loss: 110.5870
Epoch 93/1000
1/1 [==

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0251 - val_loss: 79.9445
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0230 - val_loss: 95.8596
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0251 - val_loss: 96.0368
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0247 - val_loss: 80.0092
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0291 - val_loss: 95.8298
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - val_loss: 95.8914
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0221 - val_loss: 80.1104
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0211 - val_loss: 110.6361
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0266 - val_loss: 80.1324
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0190 - val_loss: 95.8709

1/1 [==============================] - 2s 2s/step - loss: 0.0496 - val_loss: 94.3033
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0495 - val_loss: 79.2077
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0238 - val_loss: 79.2171
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0185 - val_loss: 94.9935
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0219 - val_loss: 95.1200
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0224 - val_loss: 95.0660
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0438 - val_loss: 109.7268
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0216 - val_loss: 109.8640
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0279 - val_loss: 79.2647
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0217 - val_loss: 125.5820
Epoch 256/10

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0208 - val_loss: 95.6553
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0294 - val_loss: 79.5516
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0268 - val_loss: 110.2000
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0271 - val_loss: 79.5286
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0206 - val_loss: 95.7440
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0191 - val_loss: 79.5245
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0200 - val_loss: 95.7492
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0219 - val_loss: 110.2077
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0237 - val_loss: 95.7597
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0284 - val_loss: 95.731

...

Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0365 - val_loss: 125.7437
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 109.9427
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0201 - val_loss: 79.7466
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0250 - val_loss: 65.3000
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0294 - val_loss: 95.4965
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0387 - val_loss: 95.2245
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - val_loss: 79.4691
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0198 - val_loss: 109.7174
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0210 - val_loss: 95.2540
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0411 - val_loss: 65.28

1/1 [==============================] - 2s 2s/step - loss: 0.0204 - val_loss: 109.7144
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0234 - val_loss: 79.2756
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0619 - val_loss: 94.9452
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0313 - val_loss: 109.9145
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0212 - val_loss: 79.1327
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0173 - val_loss: 109.8647
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0215 - val_loss: 125.7590
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0199 - val_loss: 79.0847
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0230 - val_loss: 94.8152
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0241 - val_loss: 94.7252
Epoch 745/1

Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0263 - val_loss: 95.0752
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0181 - val_loss: 79.7052
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0214 - val_loss: 109.5373
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0234 - val_loss: 95.0167
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0270 - val_loss: 109.5499
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0291 - val_loss: 94.8670
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0183 - val_loss: 79.4025
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 79.4286
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0328 - val_loss: 125.1885
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0182 - val_loss: 109.5

1/1 [==============================] - 2s 2s/step - loss: 0.1116 - val_loss: 109.1649
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0278 - val_loss: 109.0409
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0296 - val_loss: 78.7498
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0190 - val_loss: 78.7641
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0189 - val_loss: 78.8216
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0250 - val_loss: 78.8280
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0208 - val_loss: 78.7997
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0197 - val_loss: 78.7970
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0195 - val_loss: 64.2353
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0185 - val_loss: 78.8000
Epoch 908/100

Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0190 - val_loss: 109.2061
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0316 - val_loss: 109.1954
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0532 - val_loss: 78.7935
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0280 - val_loss: 94.8394
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0210 - val_loss: 78.7502
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0224 - val_loss: 109.0190
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0257 - val_loss: 108.9790
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0197 - val_loss: 94.8619
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0296 - val_loss: 78.9116
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0417 - val_loss: 108.

In [24]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0168 - val_loss: 79.2046
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0251 - val_loss: 109.2552
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0344 - val_loss: 79.2457
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0223 - val_loss: 109.3896
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0189 - val_loss: 109.4432
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 109.4185
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0174 - val_loss: 79.2513
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0220 - val_loss: 79.2580
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0216 - val_loss: 95.0972
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0324 - val_loss: 109.3050
Epoch 11/1000


1/1 [==============================] - 2s 2s/step - loss: 0.0328 - val_loss: 108.6387
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0168 - val_loss: 78.0253
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0274 - val_loss: 78.0797
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0497 - val_loss: 63.7446
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0590 - val_loss: 108.4925
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0216 - val_loss: 108.5837
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0205 - val_loss: 78.0850
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0180 - val_loss: 94.3325
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0215 - val_loss: 78.1843
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0279 - val_loss: 108.5517
Epoch 93/1000
1/1 [=

1/1 [==============================] - 2s 2s/step - loss: 0.0255 - val_loss: 94.9275
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0240 - val_loss: 94.9043
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - val_loss: 94.9894
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 109.2778
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0258 - val_loss: 79.0631
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0209 - val_loss: 125.3622
Epoch 171/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0332 - val_loss: 79.1274
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0389 - val_loss: 109.2620
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0188 - val_loss: 65.0028
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0197 - val_loss: 109.0500
Epoch 175/1

Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0230 - val_loss: 94.6954
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0231 - val_loss: 108.9366
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0403 - val_loss: 78.6041
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0198 - val_loss: 109.3245
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0161 - val_loss: 64.3854
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0168 - val_loss: 78.6972
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 109.5423
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0317 - val_loss: 109.4466
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0200 - val_loss: 79.0527
Epoch 256/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0170 - val_loss: 79.0

1/1 [==============================] - 2s 2s/step - loss: 0.0277 - val_loss: 95.3970
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 109.6996
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0157 - val_loss: 78.4085
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0217 - val_loss: 95.2850
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0272 - val_loss: 109.5909
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0189 - val_loss: 78.3934
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0201 - val_loss: 78.4161
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0174 - val_loss: 95.2206
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0235 - val_loss: 109.6247
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0203 - val_loss: 78.3926
Epoch 338/10

...

Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0205 - val_loss: 108.6737
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0263 - val_loss: 108.6976
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0251 - val_loss: 124.4154
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0166 - val_loss: 94.2041
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0158 - val_loss: 94.2372
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 108.5549
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0192 - val_loss: 94.2606
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0315 - val_loss: 78.4003
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0188 - val_loss: 78.3165
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0168 - val_loss: 78.3

1/1 [==============================] - 2s 2s/step - loss: 0.0222 - val_loss: 108.3138
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0181 - val_loss: 93.8224
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0175 - val_loss: 93.8587
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0235 - val_loss: 78.3549
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0451 - val_loss: 108.3923
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0301 - val_loss: 78.2751
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0178 - val_loss: 93.6904
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0151 - val_loss: 93.7179
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0172 - val_loss: 108.2139
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0220 - val_loss: 78.3148
Epoch 745/10

Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0206 - val_loss: 78.1778
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0362 - val_loss: 78.2340
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0170 - val_loss: 78.2883
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0159 - val_loss: 93.9318
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0168 - val_loss: 78.3162
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0190 - val_loss: 93.9232
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0188 - val_loss: 93.9389
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0204 - val_loss: 123.8860
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0161 - val_loss: 78.1001
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0311 - val_loss: 78.1529

1/1 [==============================] - 2s 2s/step - loss: 0.0170 - val_loss: 108.0161
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0179 - val_loss: 123.7799
Epoch 900/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0172 - val_loss: 93.9954
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 108.0144
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0178 - val_loss: 93.9413
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0154 - val_loss: 93.9814
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0157 - val_loss: 107.9862
Epoch 905/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0213 - val_loss: 92.2293
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0206 - val_loss: 78.1975
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0185 - val_loss: 78.1965
Epoch 908/1

Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0189 - val_loss: 108.0381
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0173 - val_loss: 93.7847
Epoch 982/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0167 - val_loss: 78.1341
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0163 - val_loss: 63.9259
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0255 - val_loss: 93.9506
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 78.1654
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 108.0702
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0155 - val_loss: 78.1289
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0249 - val_loss: 93.8469
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0250 - val_loss: 78.270

In [25]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0164 - val_loss: 123.7640
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0171 - val_loss: 93.8307
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0163 - val_loss: 78.0107
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0229 - val_loss: 63.8012
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0570 - val_loss: 107.7906
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0218 - val_loss: 77.7810
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0183 - val_loss: 107.8167
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0158 - val_loss: 77.8440
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0204 - val_loss: 107.8572
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0259 - val_loss: 92.0160
Epoch 11/1000
1

1/1 [==============================] - 2s 2s/step - loss: 0.0304 - val_loss: 78.0946
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0320 - val_loss: 78.3314
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0185 - val_loss: 63.9302
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0162 - val_loss: 93.5532
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0269 - val_loss: 78.3042
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0409 - val_loss: 78.2213
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0281 - val_loss: 78.1787
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0225 - val_loss: 78.2486
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0160 - val_loss: 92.4422
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0149 - val_loss: 93.9308
Epoch 93/1000
1/1 [=====

1/1 [==============================] - 2s 2s/step - loss: 0.0174 - val_loss: 93.4783
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0218 - val_loss: 93.4932
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0326 - val_loss: 93.4449
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0433 - val_loss: 77.9281
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0177 - val_loss: 77.9017
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0162 - val_loss: 93.4967
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0304 - val_loss: 107.8667
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0320 - val_loss: 93.3767
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0207 - val_loss: 93.1489
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0239 - val_loss: 93.1880
Epoch 175/1000

Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0177 - val_loss: 107.9331
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0156 - val_loss: 107.8958
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0175 - val_loss: 78.1277
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0603 - val_loss: 94.0654
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0585 - val_loss: 78.0359
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0169 - val_loss: 78.0034
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0161 - val_loss: 64.0087
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 78.0219
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0210 - val_loss: 93.8176
Epoch 256/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0325 - val_loss: 93.646

1/1 [==============================] - 2s 2s/step - loss: 0.0219 - val_loss: 93.5249
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0176 - val_loss: 93.4695
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0174 - val_loss: 107.8088
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 78.0768
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0235 - val_loss: 107.7262
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 93.5817
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0194 - val_loss: 123.1606
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0195 - val_loss: 107.8080
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0162 - val_loss: 107.6424
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 107.7129
Epoch 338

...

1/1 [==============================] - 2s 2s/step - loss: 0.0147 - val_loss: 77.4267
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0213 - val_loss: 77.3903
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0211 - val_loss: 93.2708
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0164 - val_loss: 77.4182
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0192 - val_loss: 93.1129
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0153 - val_loss: 107.0815
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0147 - val_loss: 77.4658
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0276 - val_loss: 63.5859
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0187 - val_loss: 107.1916
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0162 - val_loss: 77.5198
Epoch 664/100

Epoch 736/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0184 - val_loss: 121.8514
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0245 - val_loss: 92.6481
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0151 - val_loss: 106.6624
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0139 - val_loss: 121.9761
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0367 - val_loss: 92.4654
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0238 - val_loss: 77.0496
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0192 - val_loss: 90.8799
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0164 - val_loss: 106.3351
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0141 - val_loss: 92.4862
Epoch 745/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0163 - val_loss: 63.1

1/1 [==============================] - 2s 2s/step - loss: 0.0197 - val_loss: 93.0733
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0414 - val_loss: 106.7681
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0424 - val_loss: 106.4463
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 63.6481
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 106.7267
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 92.8486
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0220 - val_loss: 92.9653
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0159 - val_loss: 92.9694
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0192 - val_loss: 77.6011
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 77.5859
Epoch 827/10

Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0141 - val_loss: 77.7600
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 93.6828
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0167 - val_loss: 77.6465
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0212 - val_loss: 77.5700
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0240 - val_loss: 93.2436
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0310 - val_loss: 77.5652
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 107.2581
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0192 - val_loss: 107.4229
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0200 - val_loss: 93.3269
Epoch 908/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0473 - val_loss: 107.00

1/1 [==============================] - 1s 1s/step - loss: 0.0135 - val_loss: 77.2716
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0188 - val_loss: 106.7540
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0168 - val_loss: 91.2463
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 92.7211
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0144 - val_loss: 106.8814
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0197 - val_loss: 92.7042
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0336 - val_loss: 92.6762
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0285 - val_loss: 77.3306
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0270 - val_loss: 92.7235
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 92.6596
Epoch 990/100

In [26]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0176 - val_loss: 92.4623
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0205 - val_loss: 92.5892
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0151 - val_loss: 92.5719
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0138 - val_loss: 77.0689
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 77.0923
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 92.3579
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0251 - val_loss: 76.9456
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 76.9647
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 92.3136
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - val_loss: 92.3910
Epoch 11/1000
1/1 [

1/1 [==============================] - 2s 2s/step - loss: 0.0141 - val_loss: 92.6247
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0299 - val_loss: 76.8981
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0200 - val_loss: 92.5750
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0207 - val_loss: 76.9900
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0242 - val_loss: 91.3249
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0153 - val_loss: 106.8059
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0163 - val_loss: 92.5300
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0254 - val_loss: 122.4455
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0237 - val_loss: 76.9502
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0204 - val_loss: 92.5258
Epoch 93/1000
1/1 [===

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0171 - val_loss: 63.2069
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0131 - val_loss: 92.6132
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0132 - val_loss: 106.7132
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0214 - val_loss: 92.5623
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0200 - val_loss: 106.7982
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0164 - val_loss: 77.4681
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 106.7568
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0139 - val_loss: 77.4269
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0208 - val_loss: 92.3424
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0226 - val_loss: 91.62

1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 92.8266
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0151 - val_loss: 77.3987
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0181 - val_loss: 77.2731
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0149 - val_loss: 77.2756
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0154 - val_loss: 107.1417
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 92.7867
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0164 - val_loss: 91.6163
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0378 - val_loss: 92.5528
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 106.7610
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 122.1147
Epoch 256/10

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0132 - val_loss: 106.3406
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 92.4400
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0367 - val_loss: 92.4973
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0268 - val_loss: 76.7590
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0321 - val_loss: 76.8201
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0134 - val_loss: 106.4561
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - val_loss: 92.3316
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0272 - val_loss: 92.5137
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0204 - val_loss: 76.7989
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0185 - val_loss: 106.50

...

Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0326 - val_loss: 76.8243
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0207 - val_loss: 76.7798
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 76.7947
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0137 - val_loss: 105.8985
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0156 - val_loss: 105.9713
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0379 - val_loss: 91.8427
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0182 - val_loss: 106.0877
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0188 - val_loss: 76.8492
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 106.1371
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0150 - val_loss: 106.

1/1 [==============================] - 2s 2s/step - loss: 0.0139 - val_loss: 76.5982
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0150 - val_loss: 76.5224
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 91.8841
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 106.1688
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0176 - val_loss: 76.5338
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 76.5636
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0198 - val_loss: 62.6432
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0342 - val_loss: 106.1804
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0182 - val_loss: 92.1313
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 121.7727
Epoch 745/10

Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 106.5557
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 106.5608
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0174 - val_loss: 92.4115
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0198 - val_loss: 92.1864
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0293 - val_loss: 92.2840
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0169 - val_loss: 76.2904
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 76.2957
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 106.2464
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0154 - val_loss: 106.2062
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0158 - val_loss: 76.4

1/1 [==============================] - 2s 2s/step - loss: 0.0175 - val_loss: 76.3862
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0300 - val_loss: 91.6265
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0161 - val_loss: 90.5571
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 76.4109
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0177 - val_loss: 105.8082
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0456 - val_loss: 91.7577
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0167 - val_loss: 76.2160
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0207 - val_loss: 76.2299
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 90.4460
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 76.2285
Epoch 908/1000

Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 76.5650
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 121.6415
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 91.8425
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0147 - val_loss: 106.0408
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0132 - val_loss: 76.4626
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 91.7974
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0150 - val_loss: 91.8510
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0225 - val_loss: 90.7122
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0167 - val_loss: 92.0487
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0158 - val_loss: 76.451

In [27]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 76.0602
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0166 - val_loss: 61.9643
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0227 - val_loss: 91.3442
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0287 - val_loss: 91.6098
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 90.2283
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 91.5579
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 105.5953
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0151 - val_loss: 62.0969
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0259 - val_loss: 105.3829
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0161 - val_loss: 105.5532
Epoch 11/1000
1/

1/1 [==============================] - 2s 2s/step - loss: 0.0312 - val_loss: 91.8651
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 105.8763
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0138 - val_loss: 91.9028
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0154 - val_loss: 121.6317
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0212 - val_loss: 91.9111
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0244 - val_loss: 91.8344
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 91.8268
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 75.8848
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0225 - val_loss: 75.8915
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0201 - val_loss: 105.7965
Epoch 93/1000
1/1 [==

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 105.9875
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 91.9581
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0134 - val_loss: 76.0114
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0191 - val_loss: 92.0073
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 105.9246
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0227 - val_loss: 90.0221
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0232 - val_loss: 105.9680
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0175 - val_loss: 92.0956
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0118 - val_loss: 76.0361
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0122 - val_loss: 92.09

1/1 [==============================] - 1s 1s/step - loss: 0.0115 - val_loss: 75.5718
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0173 - val_loss: 75.6012
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0230 - val_loss: 91.4389
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0160 - val_loss: 75.6061
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0168 - val_loss: 91.3889
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 105.7032
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 105.7174
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0159 - val_loss: 105.7197
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0190 - val_loss: 89.9936
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0207 - val_loss: 105.4442
Epoch 256/1

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0160 - val_loss: 105.6196
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 90.1392
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 105.5621
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0199 - val_loss: 105.3513
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0281 - val_loss: 62.2492
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0210 - val_loss: 76.1821
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - val_loss: 105.4357
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 76.1811
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 105.4578
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0198 - val_loss: 76.

...

Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0248 - val_loss: 90.8934
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5236 - val_loss: 105.1491
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0486 - val_loss: 75.8796
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 91.8185
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 105.6494
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0155 - val_loss: 91.7238
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 91.6594
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0183 - val_loss: 91.7215
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0171 - val_loss: 75.8227
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 91.758

1/1 [==============================] - 2s 2s/step - loss: 0.0236 - val_loss: 105.6603
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 76.1604
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 76.1723
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0160 - val_loss: 91.2406
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0381 - val_loss: 91.2480
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 91.2222
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 105.3415
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 91.1584
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0132 - val_loss: 76.0220
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0173 - val_loss: 76.0123
Epoch 745/100

1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 76.0389
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0288 - val_loss: 91.4363
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0263 - val_loss: 89.8182
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 91.5002
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 91.4730
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0132 - val_loss: 105.2676
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 91.3459
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0158 - val_loss: 75.8128
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 91.3167
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 75.8046
Epoch 826/1000

Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 75.5522
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 61.7278
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 91.1653
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0351 - val_loss: 75.5167
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0162 - val_loss: 104.6498
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0153 - val_loss: 104.8975
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 75.7001
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 89.5394
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 91.0668
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0183 - val_loss: 91.025

1/1 [==============================] - 2s 2s/step - loss: 0.0221 - val_loss: 105.1618
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0137 - val_loss: 75.6291
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0111 - val_loss: 75.6346
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 105.1264
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 89.4535
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0167 - val_loss: 75.6381
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 105.0322
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 91.2112
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 75.5537
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 105.0500
Epoch 989/1

In [28]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 90.8457
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 75.5156
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 75.5128
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 90.8594
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 90.9235
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0322 - val_loss: 90.6510
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0248 - val_loss: 90.8436
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0111 - val_loss: 75.4500
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 90.7663
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0208 - val_loss: 120.0207
Epoch 11/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 0.0174 - val_loss: 75.7322
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0172 - val_loss: 90.9803
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0144 - val_loss: 120.2767
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 90.9754
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 104.9710
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0169 - val_loss: 120.4747
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0202 - val_loss: 104.9693
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0169 - val_loss: 75.6758
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 105.0512
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 75.6857
Epoch 93/1000
1/1 [

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 105.2540
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0212 - val_loss: 120.4689
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 104.9741
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 91.0770
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 89.6469
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0122 - val_loss: 91.0734
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 91.1908
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 91.2013
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0141 - val_loss: 91.1487
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0134 - val_loss: 91.21

1/1 [==============================] - 2s 2s/step - loss: 0.0505 - val_loss: 91.2369
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 75.6042
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 75.6242
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0120 - val_loss: 75.6004
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0150 - val_loss: 104.9700
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0144 - val_loss: 104.9906
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 91.0403
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 89.2551
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0111 - val_loss: 104.9023
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 104.8129
Epoch 256/1

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0137 - val_loss: 90.6550
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0855 - val_loss: 103.9625
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0610 - val_loss: 74.7284
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 74.7247
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0162 - val_loss: 74.7913
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0186 - val_loss: 90.4760
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 104.5699
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 104.7194
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 104.7292
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0120 - val_loss: 90.6

...

1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 90.4375
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 75.6377
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 90.3496
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0150 - val_loss: 90.3500
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0454 - val_loss: 104.7722
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0228 - val_loss: 61.2998
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 75.5891
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 104.6951
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 61.1525
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0157 - val_loss: 90.2870
Epoch 663/100

Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0161 - val_loss: 104.0248
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0149 - val_loss: 104.1861
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 90.1217
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 90.1474
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 61.1484
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0137 - val_loss: 75.1860
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0171 - val_loss: 75.1520
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0141 - val_loss: 61.1186
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 104.1231
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0131 - val_loss: 75.19

1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 104.1890
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - val_loss: 75.1223
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 90.2652
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0457 - val_loss: 75.2214
Epoch 820/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0604 - val_loss: 74.8543
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0118 - val_loss: 90.3991
Epoch 822/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0158 - val_loss: 90.5494
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1141 - val_loss: 75.5746
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2000 - val_loss: 104.3141
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0207 - val_loss: 104.2263
Epoch 826/10

Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0348 - val_loss: 74.9984
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 75.0427
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 75.0472
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 104.9579
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0402 - val_loss: 91.4204
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0147 - val_loss: 105.0973
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0134 - val_loss: 91.2278
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 75.1565
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0128 - val_loss: 104.8601
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0147 - val_loss: 91.05

1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 88.2308
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 104.0519
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 90.4378
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 104.0062
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 74.6398
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0155 - val_loss: 90.4950
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0189 - val_loss: 74.6087
Epoch 986/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0199 - val_loss: 90.5838
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0144 - val_loss: 74.5749
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 74.5997
Epoch 989/100

In [29]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0151 - val_loss: 90.6480
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 74.6300
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0171 - val_loss: 104.4472
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0111 - val_loss: 90.6673
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 74.6215
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - val_loss: 90.5816
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0263 - val_loss: 90.2333
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0179 - val_loss: 90.3267
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0150 - val_loss: 74.1577
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 74.2271
Epoch 11/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 90.0903
Epoch 84/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0124 - val_loss: 88.6380
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0440 - val_loss: 89.7993
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 104.3201
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 119.7041
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 104.2612
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0138 - val_loss: 104.3248
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 104.3598
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 74.3927
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 74.2841
Epoch 93/1000
1/1 [

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 104.3885
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 90.2281
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 104.4157
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 88.8171
Epoch 169/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0492 - val_loss: 104.1292
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0192 - val_loss: 103.8771
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 60.4240
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 89.8957
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0159 - val_loss: 104.0111
Epoch 174/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0149 - val_loss: 89.

1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 88.7723
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 103.7986
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 74.7655
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 89.8558
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 103.7126
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - val_loss: 103.8856
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 74.8627
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0169 - val_loss: 74.8750
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0153 - val_loss: 103.9530
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 74.9824
Epoch 256/1

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 103.4636
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 89.6958
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0183 - val_loss: 103.6317
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0176 - val_loss: 103.4894
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 103.4449
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 74.8002
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 74.8634
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0131 - val_loss: 74.8395
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0249 - val_loss: 89.9095
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0128 - val_loss: 74.9

...

Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0172 - val_loss: 60.4548
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 74.5042
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 104.1476
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 90.0553
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 74.6426
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0153 - val_loss: 89.9771
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 119.2262
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 89.8928
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 89.8759
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 103.82

1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 88.1550
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 104.0937
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0169 - val_loss: 73.9698
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 89.9152
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 103.9392
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 89.9775
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 103.9434
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0199 - val_loss: 103.8947
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0141 - val_loss: 73.9227
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 73.9448
Epoch 745/1

Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 103.8062
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 89.8096
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0111 - val_loss: 119.5766
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0186 - val_loss: 103.7223
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 89.8501
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 119.6232
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 74.1973
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 74.1652
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0120 - val_loss: 60.2303
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 103.

1/1 [==============================] - 2s 2s/step - loss: 0.0202 - val_loss: 103.9429
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0176 - val_loss: 103.8723
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0146 - val_loss: 89.9712
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 74.2956
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 74.2791
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0118 - val_loss: 103.9737
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 89.9136
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0324 - val_loss: 74.2625
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 60.3610
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 74.2603
Epoch 908/10

Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 90.0314
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0132 - val_loss: 74.4662
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 104.1438
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0263 - val_loss: 103.6579
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 89.9407
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 103.5675
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 103.7764
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 103.8295
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0186 - val_loss: 74.0724
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0211 - val_loss: 89.

In [30]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 89.5851
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 89.6068
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 89.6207
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 74.0563
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0125 - val_loss: 60.2867
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 103.4245
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 74.0223
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 89.6320
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 89.6989
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 74.0693
Epoch 11/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 0.0097 - val_loss: 74.0967
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 74.1345
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 103.3971
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0137 - val_loss: 87.8698
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0234 - val_loss: 74.1425
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0216 - val_loss: 89.4860
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 74.2819
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 89.6081
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 103.4322
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 89.6601
Epoch 93/1000
1/1 [===

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 89.7811
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0209 - val_loss: 89.6066
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 103.5049
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 74.0834
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 74.0709
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 89.7176
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 73.9856
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 89.6775
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 74.0200
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 103.394

1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 89.4859
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 74.3061
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 74.2938
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 103.2577
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0128 - val_loss: 74.3001
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 118.5456
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 103.3237
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0164 - val_loss: 74.3156
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1736 - val_loss: 73.4241
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0252 - val_loss: 89.3852
Epoch 256/10

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0756 - val_loss: 89.8440
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 60.8536
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 89.8904
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - val_loss: 89.7192
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0611 - val_loss: 103.4511
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0501 - val_loss: 89.8172
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0128 - val_loss: 89.8673
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 88.0493
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0122 - val_loss: 89.9140
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 103.880

...

Epoch 654/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0323 - val_loss: 103.0916
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 74.1720
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 118.2843
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 89.3175
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 74.1254
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 89.3667
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 103.2954
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0206 - val_loss: 74.2030
Epoch 662/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0184 - val_loss: 74.2731
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 89.39

1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 103.9453
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0097 - val_loss: 74.9004
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 103.9532
Epoch 738/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0126 - val_loss: 103.9516
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 103.8340
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 74.8591
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 74.8549
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 103.8241
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 89.3923
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0147 - val_loss: 89.3207
Epoch 745/

Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0234 - val_loss: 103.3502
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0222 - val_loss: 103.0528
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 89.0215
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 89.0842
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0182 - val_loss: 74.4526
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 74.4115
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 103.1654
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 117.6396
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 103.1413
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 89.

1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 89.1433
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 89.2292
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 102.7796
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0156 - val_loss: 102.8661
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 73.8655
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 102.8726
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 73.8167
Epoch 905/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0152 - val_loss: 73.7940
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0138 - val_loss: 73.8234
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 88.9673
Epoch 908/10

Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 89.4794
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 89.4442
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0154 - val_loss: 102.8924
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0218 - val_loss: 103.1505
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 74.0315
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0143 - val_loss: 103.0820
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 73.9598
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 89.4631
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 102.8896
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0147 - val_loss: 118.

In [31]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 73.7098
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 102.3547
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 102.3523
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 73.8307
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 89.1993
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0175 - val_loss: 102.4214
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 117.8012
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 89.1196
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 73.9033
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0159 - val_loss: 73.8363
Epoch 11/1000
1

1/1 [==============================] - 2s 2s/step - loss: 0.0206 - val_loss: 102.9024
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 102.9401
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 88.9451
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0094 - val_loss: 74.1245
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 74.1444
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0138 - val_loss: 74.1119
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0197 - val_loss: 102.8364
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0150 - val_loss: 102.5911
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 74.0550
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0097 - val_loss: 102.6984
Epoch 93/1000
1/1 [

1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 73.6144
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 102.7385
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0195 - val_loss: 102.5229
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 73.6954
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 73.6953
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 88.9775
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 73.7334
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0118 - val_loss: 102.7364
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 73.7348
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 73.6672
Epoch 175/10

1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 89.6451
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 74.2486
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 103.4572
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0094 - val_loss: 89.4922
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 74.2985
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 89.5121
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 74.2283
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 103.4212
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0097 - val_loss: 74.2351
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 103.2541
Epoch 256/10

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0134 - val_loss: 88.0908
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 89.8089
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 74.4138
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 103.4561
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 74.3820
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 103.2577
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0393 - val_loss: 89.7834
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0254 - val_loss: 74.2487
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 102.9570
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 118.3

...

Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 88.7833
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 73.7102
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0094 - val_loss: 102.5609
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 73.7207
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0154 - val_loss: 88.9708
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 73.6234
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 73.6221
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 102.9112
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 73.6817
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 102.79

1/1 [==============================] - 2s 2s/step - loss: 0.5037 - val_loss: 101.9221
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 88.8458
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0118 - val_loss: 88.8853
Epoch 738/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0088 - val_loss: 88.9401
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0154 - val_loss: 74.4693
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0124 - val_loss: 102.3542
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0122 - val_loss: 102.1490
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0128 - val_loss: 87.9510
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0097 - val_loss: 88.8359
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 88.8198
Epoch 745/10

Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 117.2513
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 102.3803
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 88.8357
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 88.6805
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 88.7509
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 73.8514
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0152 - val_loss: 102.4501
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0169 - val_loss: 102.2843
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 73.7131
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 102.

1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 117.5861
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 88.5842
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0091 - val_loss: 88.6346
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 87.0688
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0198 - val_loss: 73.3627
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0270 - val_loss: 101.9838
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0122 - val_loss: 102.0327
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 73.2186
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0091 - val_loss: 73.2591
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 117.3675
Epoch 908/1

Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 102.1217
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 73.4261
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 59.6066
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 102.0797
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 88.3974
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 102.1366
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 102.3399
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 88.3445
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0094 - val_loss: 116.9348
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 88.

In [32]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0201 - val_loss: 73.4590
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 102.0679
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 59.5919
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 88.2948
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 102.0913
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0135 - val_loss: 87.1331
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 73.2732
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 102.1455
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 73.2752
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 88.2530
Epoch 11/1000
1/

1/1 [==============================] - 2s 2s/step - loss: 0.0091 - val_loss: 102.4918
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 88.5085
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 102.5552
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0127 - val_loss: 102.4151
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 59.7176
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 73.5795
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0138 - val_loss: 102.7169
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 59.5954
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 88.5871
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 73.5391
Epoch 93/1000
1/1 [=

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 59.5286
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 102.3140
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0118 - val_loss: 88.5150
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 117.5220
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0159 - val_loss: 102.2284
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 88.4317
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 88.3752
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0091 - val_loss: 102.1091
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0120 - val_loss: 102.2882
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 88.

1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 102.3711
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0118 - val_loss: 102.4227
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 88.5810
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 59.7020
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 73.4742
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 73.4426
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0305 - val_loss: 102.3892
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 73.5367
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 88.4011
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 88.4465
Epoch 256/10

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 102.1954
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 88.5939
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0139 - val_loss: 88.4689
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0094 - val_loss: 73.4467
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 88.4677
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0081 - val_loss: 102.2206
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 73.4410
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 73.4996
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 73.4872
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 88.432

...

Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0114 - val_loss: 88.4177
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 102.1807
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 73.4808
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0094 - val_loss: 73.4163
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 88.5674
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 88.4000
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0239 - val_loss: 73.4970
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 73.5683
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 88.5730
Epoch 663/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 102.133

1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 87.3410
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 73.4420
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 101.9969
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 73.3839
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 73.3829
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0091 - val_loss: 88.3957
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 102.1035
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0128 - val_loss: 73.4255
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0192 - val_loss: 88.7361
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 87.0526
Epoch 745/100

Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 73.1335
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 73.0819
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 87.9728
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0131 - val_loss: 101.6906
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 87.8890
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 101.6347
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 101.7080
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 73.0974
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 86.7930
Epoch 826/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 73.08

1/1 [==============================] - 2s 2s/step - loss: 0.0141 - val_loss: 87.9606
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 72.7912
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 116.4640
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 87.8981
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 72.8846
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0155 - val_loss: 116.7185
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 72.8313
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 101.7519
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 72.8693
Epoch 907/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 101.6609
Epoch 908/1

Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0094 - val_loss: 101.8075
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0097 - val_loss: 101.8517
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 101.6913
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 73.2737
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 86.9160
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 87.9118
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 101.6872
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 73.2293
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0170 - val_loss: 73.2080
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - val_loss: 101.

In [33]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 73.1096
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 86.7364
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 101.5773
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 87.7677
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0091 - val_loss: 101.4459
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 73.0932
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 73.0854
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 116.0282
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 87.8285
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 87.8461
Epoch 11/1000
1/

1/1 [==============================] - 2s 2s/step - loss: 0.0110 - val_loss: 86.8558
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 87.7585
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0080 - val_loss: 87.7522
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 116.0287
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 72.9629
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 101.1826
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 59.3290
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 101.2385
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0363 - val_loss: 72.8809
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0317 - val_loss: 86.5792
Epoch 93/1000
1/1 [==

Epoch 165/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 72.9522
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 72.9990
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 72.9772
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 72.9425
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 101.6633
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0242 - val_loss: 72.9184
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0119 - val_loss: 101.5558
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 72.8602
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 87.7645
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 87.756

1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 72.6204
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0123 - val_loss: 72.5574
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 72.5611
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 72.6060
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 72.5392
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 72.5378
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0081 - val_loss: 101.4889
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 87.4654
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0148 - val_loss: 86.6234
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 87.6122
Epoch 256/1000

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0131 - val_loss: 72.5056
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 87.5369
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 87.5211
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 101.2180
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 101.1780
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0170 - val_loss: 87.6062
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 87.5438
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 101.4145
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 72.4903
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 87.54

...

1/1 [==============================] - 2s 2s/step - loss: 0.0078 - val_loss: 101.0277
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 72.3836
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 101.0779
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 87.5086
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 72.3420
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 100.9634
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0098 - val_loss: 85.9099
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0122 - val_loss: 87.5054
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0227 - val_loss: 101.0421
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0129 - val_loss: 101.1109
Epoch 663/

Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 72.6665
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0077 - val_loss: 72.6426
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 86.6446
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - val_loss: 87.4212
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0105 - val_loss: 87.4250
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 72.6562
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0080 - val_loss: 72.6263
Epoch 742/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 72.6044
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 116.0523
Epoch 744/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 101.315

1/1 [==============================] - 2s 2s/step - loss: 0.0185 - val_loss: 100.9686
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0205 - val_loss: 101.1494
Epoch 818/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0095 - val_loss: 116.0757
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 101.1017
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0081 - val_loss: 101.0837
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0130 - val_loss: 58.5990
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0156 - val_loss: 72.3750
Epoch 823/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 72.3797
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 115.7444
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 87.1959
Epoch 826

1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 72.3352
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 87.4403
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0116 - val_loss: 101.1497
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 101.0726
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 87.5113
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 87.4627
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0081 - val_loss: 72.3762
Epoch 904/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 101.1403
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0140 - val_loss: 58.7204
Epoch 906/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 72.3534
Epoch 907/10

Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 87.2207
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0099 - val_loss: 58.3449
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0097 - val_loss: 72.1755
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 72.2413
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 72.2077
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0200 - val_loss: 101.4337
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0628 - val_loss: 101.2578
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 58.2005
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 101.3827
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 87.46

In [34]:
# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1000,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0109 - val_loss: 101.0840
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0080 - val_loss: 101.0314
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 72.0127
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 116.3125
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 72.0626
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 87.3545
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 87.3844
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0080 - val_loss: 100.9933
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 101.2299
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0111 - val_loss: 58.3676
Epoch 11/1000


1/1 [==============================] - 2s 2s/step - loss: 0.0177 - val_loss: 101.0947
Epoch 84/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0101 - val_loss: 101.2754
Epoch 85/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 87.5646
Epoch 86/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 72.3922
Epoch 87/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0077 - val_loss: 87.4506
Epoch 88/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 116.2309
Epoch 89/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 87.4656
Epoch 90/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 101.1167
Epoch 91/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0083 - val_loss: 116.2213
Epoch 92/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0080 - val_loss: 101.1785
Epoch 93/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 88.2628
Epoch 166/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 117.2241
Epoch 167/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 88.1077
Epoch 168/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0133 - val_loss: 101.7133
Epoch 169/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0091 - val_loss: 101.6718
Epoch 170/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 101.7906
Epoch 171/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0134 - val_loss: 101.5201
Epoch 172/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0117 - val_loss: 59.0380
Epoch 173/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 72.4815
Epoch 174/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 101.6342
Epoch 175

1/1 [==============================] - 2s 2s/step - loss: 0.0100 - val_loss: 87.4888
Epoch 247/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0077 - val_loss: 72.3962
Epoch 248/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 101.1171
Epoch 249/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 72.4076
Epoch 250/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0126 - val_loss: 87.3738
Epoch 251/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0158 - val_loss: 101.1143
Epoch 252/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0108 - val_loss: 87.4710
Epoch 253/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 85.9285
Epoch 254/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0080 - val_loss: 100.9683
Epoch 255/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0082 - val_loss: 87.4806
Epoch 256/10

Epoch 328/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 72.2468
Epoch 329/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 87.3735
Epoch 330/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0081 - val_loss: 87.2996
Epoch 331/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0077 - val_loss: 58.5709
Epoch 332/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 100.9275
Epoch 333/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 87.1924
Epoch 334/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 100.8786
Epoch 335/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 72.2355
Epoch 336/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 100.8808
Epoch 337/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 87.21

...

1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 87.0680
Epoch 654/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0076 - val_loss: 100.6633
Epoch 655/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 100.6648
Epoch 656/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0196 - val_loss: 87.1021
Epoch 657/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0113 - val_loss: 72.4767
Epoch 658/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 72.4179
Epoch 659/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0077 - val_loss: 100.7070
Epoch 660/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0084 - val_loss: 72.4365
Epoch 661/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0112 - val_loss: 58.9467
Epoch 662/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0092 - val_loss: 87.0640
Epoch 663/10

1/1 [==============================] - 2s 2s/step - loss: 0.0104 - val_loss: 86.6653
Epoch 735/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0136 - val_loss: 72.3184
Epoch 736/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 72.2444
Epoch 737/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0103 - val_loss: 72.2714
Epoch 738/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 72.3079
Epoch 739/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 72.2849
Epoch 740/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0142 - val_loss: 86.9402
Epoch 741/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 72.2855
Epoch 742/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0084 - val_loss: 58.5596
Epoch 743/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0074 - val_loss: 72.2916
Epoch 744/1000


Epoch 816/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0078 - val_loss: 100.5253
Epoch 817/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0086 - val_loss: 100.5833
Epoch 818/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0092 - val_loss: 72.3895
Epoch 819/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0156 - val_loss: 86.6900
Epoch 820/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0089 - val_loss: 85.7338
Epoch 821/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0077 - val_loss: 100.3519
Epoch 822/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0080 - val_loss: 72.2376
Epoch 823/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0101 - val_loss: 100.3864
Epoch 824/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0107 - val_loss: 72.1575
Epoch 825/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0087 - val_loss: 100.

1/1 [==============================] - 2s 2s/step - loss: 0.0165 - val_loss: 100.4983
Epoch 898/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 100.4278
Epoch 899/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 100.5586
Epoch 900/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0078 - val_loss: 72.0762
Epoch 901/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 72.0827
Epoch 902/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0206 - val_loss: 100.7312
Epoch 903/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0121 - val_loss: 72.0787
Epoch 904/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0082 - val_loss: 85.5367
Epoch 905/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0085 - val_loss: 87.0519
Epoch 906/1000
1/1 [==============================] - 1s 1s/step - loss: 0.0100 - val_loss: 100.3538
Epoch 907/

Epoch 979/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 87.0562
Epoch 980/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0173 - val_loss: 100.7248
Epoch 981/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0093 - val_loss: 87.0314
Epoch 982/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0102 - val_loss: 58.3496
Epoch 983/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0078 - val_loss: 72.1098
Epoch 984/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0106 - val_loss: 86.9677
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0096 - val_loss: 100.7394
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0088 - val_loss: 87.0036
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0090 - val_loss: 100.6429
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - val_loss: 72.07